# LGBM gpu 세팅

In [ ]:
!git clone https://github.com/Microsoft/LightGBM

Cloning into 'LightGBM'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 20667 (delta 6), reused 4 (delta 1), pack-reused 20646
Receiving objects: 100% (20667/20667), 16.06 MiB | 30.41 MiB/s, done.
Resolving deltas: 100% (15083/15083), done.


In [ ]:
cd LightGBM

/content/LightGBM/python-package/LightGBM


In [ ]:
!mkdir build

In [ ]:
!cmake -DUSE_GPU=1

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
Traceback (most recent call last):
  File "/usr/local/bin/cmake", line 8, in <module>
    sys.exit(cmake())
  File "/usr/local/lib/python2.7/dist-packages/cmake/__init__.py", line 34, in cmake
    raise SystemExit(_program('cmake', sys.argv[1:]))



In [ ]:
!make -j$(nproc)

make: *** No targets specified and no makefile found.  Stop.


In [ ]:
!sudo apt-get -y install python-pip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-pip is already the newest version (9.0.1-2.3~ubuntu1.18.04.4).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [ ]:
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (50.3.2)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.1.4)
Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.19.4)
Requirement already up-to-date: scipy in /usr/local/lib/python3.6/dist-packages (1.5.4)
  Using cached https://files.pythonhosted.org/packages/5c/a1/273def87037a7fb010512bbc5901c31cfddfca8080bc63b42b26e3cc55b3/scikit_learn-0.23.2-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: scikit-learn 0.21.3
    Uninstalling scikit-learn-0.21.3:
      Successfully uninstalled scikit-learn-0.21.3


In [ ]:
%cd /content/LightGBM/python-package

/content/LightGBM/python-package


In [ ]:
!sudo python setup.py install

running install
removing '/content/LightGBM/python-package/compile/include' (and everything under it)
removing '/content/LightGBM/python-package/compile/src' (and everything under it)
removing '/content/LightGBM/python-package/build_cpp' (and everything under it)
INFO:LightGBM:Starting to compile the library.
INFO:LightGBM:Starting to compile with CMake.
interrupted


In [ ]:
!pip3 uninstall scikit-learn

Uninstalling scikit-learn-0.23.2:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/scikit_learn-0.23.2.dist-info/*
    /usr/local/lib/python3.6/dist-packages/scikit_learn.libs/libgomp-3300acd3.so.1.0.0
    /usr/local/lib/python3.6/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-0.23.2


In [ ]:
!pip3 install scikit-learn==0.21.3

  Using cached https://files.pythonhosted.org/packages/a0/c5/d2238762d780dde84a20b8c761f563fe882b88c5a5fb03c056547c442a19/scikit_learn-0.21.3-cp36-cp36m-manylinux1_x86_64.whl
ERROR: pycaret 2.2.2 has requirement scikit-learn>=0.23.2, but you'll have scikit-learn 0.21.3 which is incompatible.
ERROR: imbalanced-learn 0.7.0 has requirement scikit-learn>=0.23, but you'll have scikit-learn 0.21.3 which is incompatible.


In [ ]:
#버전확인
!pip list | grep scikit-learn

scikit-learn                  0.21.3         


# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import os

In [ ]:
speech = pd.read_csv('/content/drive/My Drive/MLtutoring/Final/pd_speech_features_modified.csv')

In [ ]:
speech['class'].value_counts()

1    564
0    192
Name: class, dtype: int64

## Gender를 category로 인식시키기 위해 one-hot encoding(새로운 df 다루기 전 실행 필수!!)

In [ ]:
df = speech.copy()
df

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,rapJitter,ppq5Jitter,ddpJitter,locShimmer,locDbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer,meanAutoCorrHarmonicity,meanNoiseToHarmHarmonicity,meanHarmToNoiseHarmonicity,minIntensity,maxIntensity,meanIntensity,f1,f2,f3,f4,b1,b2,b3,b4,GQ_prc5_95,GQ_std_cycle_open,GQ_std_cycle_closed,GNE_mean,GNE_std,GNE_SNR_TKEO,...,tqwt_skewnessValue_dec_34,tqwt_skewnessValue_dec_35,tqwt_skewnessValue_dec_36,tqwt_kurtosisValue_dec_1,tqwt_kurtosisValue_dec_2,tqwt_kurtosisValue_dec_3,tqwt_kurtosisValue_dec_4,tqwt_kurtosisValue_dec_5,tqwt_kurtosisValue_dec_6,tqwt_kurtosisValue_dec_7,tqwt_kurtosisValue_dec_8,tqwt_kurtosisValue_dec_9,tqwt_kurtosisValue_dec_10,tqwt_kurtosisValue_dec_11,tqwt_kurtosisValue_dec_12,tqwt_kurtosisValue_dec_13,tqwt_kurtosisValue_dec_14,tqwt_kurtosisValue_dec_15,tqwt_kurtosisValue_dec_16,tqwt_kurtosisValue_dec_17,tqwt_kurtosisValue_dec_18,tqwt_kurtosisValue_dec_19,tqwt_kurtosisValue_dec_20,tqwt_kurtosisValue_dec_21,tqwt_kurtosisValue_dec_22,tqwt_kurtosisValue_dec_23,tqwt_kurtosisValue_dec_24,tqwt_kurtosisValue_dec_25,tqwt_kurtosisValue_dec_26,tqwt_kurtosisValue_dec_27,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,0.000018,0.00067,0.00129,0.00200,0.05883,0.517,0.03011,0.03496,0.04828,0.09034,0.970805,0.036223,18.995,69.997496,76.088046,72.465512,539.342735,1031.849040,2447.162183,3655.054806,101.092218,83.147440,255.214830,396.643631,0.77778,11.7245,2.8277,1.17300,0.265120,0.083127,...,0.071728,0.010352,-2.730300,66.5007,36.9934,26.3508,51.9577,21.5451,8.2488,8.0024,6.9635,6.9189,5.0622,4.6068,7.7218,2.7224,2.4171,2.9383,4.2077,3.1541,2.8531,2.7496,2.1550,2.9457,2.1993,1.9830,1.8314,2.0062,1.6058,1.5466,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,0.000016,0.00052,0.00112,0.00157,0.05516,0.502,0.02320,0.03675,0.06195,0.06961,0.984322,0.017974,21.497,67.415903,73.046374,71.528945,564.363614,1016.367294,2383.565201,3498.681572,58.465428,86.487292,248.357127,218.229722,0.81250,13.8284,2.8908,1.02210,0.220040,0.127410,...,0.729330,0.780410,5.229400,8643.9860,3962.0554,2976.2411,4329.0607,4005.1329,1127.6762,116.5331,20.2332,13.6395,12.5514,13.6965,60.8974,6.2958,3.0323,3.4039,4.3794,2.4329,2.0585,2.1839,2.2061,3.0999,1.9824,1.6227,1.5783,2.0470,1.5772,1.5530,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,0.000015,0.00057,0.00111,0.00171,0.09902,0.897,0.05094,0.06497,0.07772,0.15282,0.974846,0.026313,17.651,62.661706,71.633549,68.086583,548.444604,1032.406341,2357.826954,3678.128717,160.387771,54.685168,151.694847,84.240339,0.81818,26.9273,2.6975,0.84951,0.157560,0.116890,...,0.269510,-0.005522,0.350540,29.2717,32.4971,38.9453,85.1480,62.5132,9.7308,6.8890,6.1357,5.4926,3.9078,4.0864,6.9828,3.4411,3.5173,3.7204,4.2212,3.4881,3.4851,3.3007,2.0427,3.1436,2.1203,1.6627,1.6731,3.2597,1.5921,1.5399,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,0.000046,0.00149,0.00268,0.00446,0.05451,0.527,0.02395,0.02857,0.04462,0.07185,0.968343,0.042003,19.865,76.306989,81.000749,79.190593,819.529588,1201.813897,3154.035654,4122.163933,238.667052,191.984916,573.752909,526.147599,0.98548,139.5744,1.6961,0.83405,0.172950,0.147370,...,0.366920,-0.492650,0.191640,591.0116,65.3827,53.9852,45.4458,34.6650,88.3259,46.6869,23.2911,19.2022,42.3957,100.3649,7.1967,6.3892,6.5496,6.0264,4.7656,4.8909,4.2531,3.0295,2.0362,1.8478,2.5776,2.2064,1.9491,1.9120,1.8829,6.9761,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4

In [ ]:
#남녀 한번 세어보자
df['gender'].value_counts()

1    390
0    366
Name: gender, dtype: int64

In [ ]:
gender = pd.get_dummies(df.loc[:, 'gender'])
# pd.get_dummies(df)# 원핫인코딩 실행

In [ ]:
gender = gender.rename(columns={0: 'female', 1: 'male'})
gender

,female,male
0,0,1
1,0,1
2,0,1
3,1,0
4,1,0
...,...,...
751,1,0
752,1,0
753,1,0
754,1,0


In [ ]:
df = pd.concat([gender, df], axis=1)
df

,female,male,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,rapJitter,ppq5Jitter,ddpJitter,locShimmer,locDbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer,meanAutoCorrHarmonicity,meanNoiseToHarmHarmonicity,meanHarmToNoiseHarmonicity,minIntensity,maxIntensity,meanIntensity,f1,f2,f3,f4,b1,b2,b3,b4,GQ_prc5_95,GQ_std_cycle_open,GQ_std_cycle_closed,GNE_mean,...,tqwt_skewnessValue_dec_34,tqwt_skewnessValue_dec_35,tqwt_skewnessValue_dec_36,tqwt_kurtosisValue_dec_1,tqwt_kurtosisValue_dec_2,tqwt_kurtosisValue_dec_3,tqwt_kurtosisValue_dec_4,tqwt_kurtosisValue_dec_5,tqwt_kurtosisValue_dec_6,tqwt_kurtosisValue_dec_7,tqwt_kurtosisValue_dec_8,tqwt_kurtosisValue_dec_9,tqwt_kurtosisValue_dec_10,tqwt_kurtosisValue_dec_11,tqwt_kurtosisValue_dec_12,tqwt_kurtosisValue_dec_13,tqwt_kurtosisValue_dec_14,tqwt_kurtosisValue_dec_15,tqwt_kurtosisValue_dec_16,tqwt_kurtosisValue_dec_17,tqwt_kurtosisValue_dec_18,tqwt_kurtosisValue_dec_19,tqwt_kurtosisValue_dec_20,tqwt_kurtosisValue_dec_21,tqwt_kurtosisValue_dec_22,tqwt_kurtosisValue_dec_23,tqwt_kurtosisValue_dec_24,tqwt_kurtosisValue_dec_25,tqwt_kurtosisValue_dec_26,tqwt_kurtosisValue_dec_27,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,0.000018,0.00067,0.00129,0.00200,0.05883,0.517,0.03011,0.03496,0.04828,0.09034,0.970805,0.036223,18.995,69.997496,76.088046,72.465512,539.342735,1031.849040,2447.162183,3655.054806,101.092218,83.147440,255.214830,396.643631,0.77778,11.7245,2.8277,1.17300,...,0.071728,0.010352,-2.730300,66.5007,36.9934,26.3508,51.9577,21.5451,8.2488,8.0024,6.9635,6.9189,5.0622,4.6068,7.7218,2.7224,2.4171,2.9383,4.2077,3.1541,2.8531,2.7496,2.1550,2.9457,2.1993,1.9830,1.8314,2.0062,1.6058,1.5466,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,0.000016,0.00052,0.00112,0.00157,0.05516,0.502,0.02320,0.03675,0.06195,0.06961,0.984322,0.017974,21.497,67.415903,73.046374,71.528945,564.363614,1016.367294,2383.565201,3498.681572,58.465428,86.487292,248.357127,218.229722,0.81250,13.8284,2.8908,1.02210,...,0.729330,0.780410,5.229400,8643.9860,3962.0554,2976.2411,4329.0607,4005.1329,1127.6762,116.5331,20.2332,13.6395,12.5514,13.6965,60.8974,6.2958,3.0323,3.4039,4.3794,2.4329,2.0585,2.1839,2.2061,3.0999,1.9824,1.6227,1.5783,2.0470,1.5772,1.5530,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,0.000015,0.00057,0.00111,0.00171,0.09902,0.897,0.05094,0.06497,0.07772,0.15282,0.974846,0.026313,17.651,62.661706,71.633549,68.086583,548.444604,1032.406341,2357.826954,3678.128717,160.387771,54.685168,151.694847,84.240339,0.81818,26.9273,2.6975,0.84951,...,0.269510,-0.005522,0.350540,29.2717,32.4971,38.9453,85.1480,62.5132,9.7308,6.8890,6.1357,5.4926,3.9078,4.0864,6.9828,3.4411,3.5173,3.7204,4.2212,3.4881,3.4851,3.3007,2.0427,3.1436,2.1203,1.6627,1.6731,3.2597,1.5921,1.5399,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,0.000046,0.00149,0.00268,0.00446,0.05451,0.527,0.02395,0.02857,0.04462,0.07185,0.968343,0.042003,19.865,76.306989,81.000749,79.190593,819.529588,1201.813897,3154.035654,4122.163933,238.667052,191.984916,573.752909,526.147599,0.98548,139.5744,1.6961,0.83405,...,0.366920,-0.492650,0.191640,591.0116,65.3827,53.9852,45.4458,34.6650,88.3259,46.6869,23.2911,19.2022,42.3957,100.3649,7.1967,6.3892,6.5496,6.0264,4.7656,4.8909,4.2531,3.0295,2.0362,1.8478,2.5776,2.2064,1.9491,1.9120,1.8829,6.9761,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,1,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.0053

In [ ]:
df = df.drop(columns=['gender'])
df

,female,male,id,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,rapJitter,ppq5Jitter,ddpJitter,locShimmer,locDbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer,meanAutoCorrHarmonicity,meanNoiseToHarmHarmonicity,meanHarmToNoiseHarmonicity,minIntensity,maxIntensity,meanIntensity,f1,f2,f3,f4,b1,b2,b3,b4,GQ_prc5_95,GQ_std_cycle_open,GQ_std_cycle_closed,GNE_mean,GNE_std,...,tqwt_skewnessValue_dec_34,tqwt_skewnessValue_dec_35,tqwt_skewnessValue_dec_36,tqwt_kurtosisValue_dec_1,tqwt_kurtosisValue_dec_2,tqwt_kurtosisValue_dec_3,tqwt_kurtosisValue_dec_4,tqwt_kurtosisValue_dec_5,tqwt_kurtosisValue_dec_6,tqwt_kurtosisValue_dec_7,tqwt_kurtosisValue_dec_8,tqwt_kurtosisValue_dec_9,tqwt_kurtosisValue_dec_10,tqwt_kurtosisValue_dec_11,tqwt_kurtosisValue_dec_12,tqwt_kurtosisValue_dec_13,tqwt_kurtosisValue_dec_14,tqwt_kurtosisValue_dec_15,tqwt_kurtosisValue_dec_16,tqwt_kurtosisValue_dec_17,tqwt_kurtosisValue_dec_18,tqwt_kurtosisValue_dec_19,tqwt_kurtosisValue_dec_20,tqwt_kurtosisValue_dec_21,tqwt_kurtosisValue_dec_22,tqwt_kurtosisValue_dec_23,tqwt_kurtosisValue_dec_24,tqwt_kurtosisValue_dec_25,tqwt_kurtosisValue_dec_26,tqwt_kurtosisValue_dec_27,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,0.000018,0.00067,0.00129,0.00200,0.05883,0.517,0.03011,0.03496,0.04828,0.09034,0.970805,0.036223,18.995,69.997496,76.088046,72.465512,539.342735,1031.849040,2447.162183,3655.054806,101.092218,83.147440,255.214830,396.643631,0.77778,11.7245,2.8277,1.17300,0.265120,...,0.071728,0.010352,-2.730300,66.5007,36.9934,26.3508,51.9577,21.5451,8.2488,8.0024,6.9635,6.9189,5.0622,4.6068,7.7218,2.7224,2.4171,2.9383,4.2077,3.1541,2.8531,2.7496,2.1550,2.9457,2.1993,1.9830,1.8314,2.0062,1.6058,1.5466,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,0.000016,0.00052,0.00112,0.00157,0.05516,0.502,0.02320,0.03675,0.06195,0.06961,0.984322,0.017974,21.497,67.415903,73.046374,71.528945,564.363614,1016.367294,2383.565201,3498.681572,58.465428,86.487292,248.357127,218.229722,0.81250,13.8284,2.8908,1.02210,0.220040,...,0.729330,0.780410,5.229400,8643.9860,3962.0554,2976.2411,4329.0607,4005.1329,1127.6762,116.5331,20.2332,13.6395,12.5514,13.6965,60.8974,6.2958,3.0323,3.4039,4.3794,2.4329,2.0585,2.1839,2.2061,3.0999,1.9824,1.6227,1.5783,2.0470,1.5772,1.5530,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,0.000015,0.00057,0.00111,0.00171,0.09902,0.897,0.05094,0.06497,0.07772,0.15282,0.974846,0.026313,17.651,62.661706,71.633549,68.086583,548.444604,1032.406341,2357.826954,3678.128717,160.387771,54.685168,151.694847,84.240339,0.81818,26.9273,2.6975,0.84951,0.157560,...,0.269510,-0.005522,0.350540,29.2717,32.4971,38.9453,85.1480,62.5132,9.7308,6.8890,6.1357,5.4926,3.9078,4.0864,6.9828,3.4411,3.5173,3.7204,4.2212,3.4881,3.4851,3.3007,2.0427,3.1436,2.1203,1.6627,1.6731,3.2597,1.5921,1.5399,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,1,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,0.000046,0.00149,0.00268,0.00446,0.05451,0.527,0.02395,0.02857,0.04462,0.07185,0.968343,0.042003,19.865,76.306989,81.000749,79.190593,819.529588,1201.813897,3154.035654,4122.163933,238.667052,191.984916,573.752909,526.147599,0.98548,139.5744,1.6961,0.83405,0.172950,...,0.366920,-0.492650,0.191640,591.0116,65.3827,53.9852,45.4458,34.6650,88.3259,46.6869,23.2911,19.2022,42.3957,100.3649,7.1967,6.3892,6.5496,6.0264,4.7656,4.8909,4.2531,3.0295,2.0362,1.8478,2.5776,2.2064,1.9491,1.9120,1.8829,6.9761,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,1,0.32790,0.79782,0.53028,236,2

## 10 fold 만들고 데이터 처리

In [ ]:
# StratifiedKFold guerantees uniform proportion of target values for each fold
from sklearn.model_selection import StratifiedKFold

RS = 456
# shuffle: random shuffle data, random_state: key of random shuffle
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=RS)

In [ ]:
target_col = 'class'
unused_cols = []

In [ ]:
X = df[[col for col in df.columns if col not in unused_cols + [target_col]]].values
y = df[target_col].values

## LightGradientBoostingMachine: 0.9168

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score  # for printing score

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
accs = []
aucs = []

model = LGBMClassifier(random_state=RS, device='gpu')

for fold, (train_indice, test_indice) in enumerate(kfold.split(X, y)):
    X_train = X[train_indice]
    y_train = y[train_indice]
    X_test = X[test_indice]
    y_test = y[test_indice]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    accs.append(acc)
    aucs.append(auc)
    print("[Fold {}] AUC : {}".format(fold, aucs[fold]))

print('Mean accuracy : {}'.format(np.mean(accs)))
print('Mean AUC : {}'.format(np.means(aucs))) 

[Fold 0] accuracy : 0.8701298701298701
[Fold 1] accuracy : 0.8961038961038961
[Fold 2] accuracy : 0.9078947368421053
[Fold 3] accuracy : 0.9210526315789473
[Fold 4] accuracy : 0.9066666666666666
[Fold 5] accuracy : 0.9066666666666666
[Fold 6] accuracy : 0.8933333333333333
[Fold 7] accuracy : 0.9466666666666667
[Fold 8] accuracy : 0.96
[Fold 9] accuracy : 0.96
Mean accuracy : 0.9168514467988154


그냥 해도 정확도 0.9167(RS=123), 0.9168(RS=456).
그런데 칼럼이 너무 많으니까 줄여보자

## XGBoost: 0.8415

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score  # for printing score
# from sklearn.model_selection import GridSearchCV

In [ ]:
from xgboost import XGBRFClassifier

In [ ]:
accs = []

model = XGBRFClassifier(random_state=RS)

for fold, (train_indice, test_indice) in enumerate(kfold.split(X, y)):
    X_train = X[train_indice]
    y_train = y[train_indice]
    X_test = X[test_indice]
    y_test = y[test_indice]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("[Fold {}] accuracy : {}".format(fold, accuracy_score(y_test, y_pred)))
    accs.append(acc)

print('Mean accuracy : {}'.format(np.mean(accs)))     

[Fold 0] accuracy : 0.7662337662337663
[Fold 1] accuracy : 0.8181818181818182
[Fold 2] accuracy : 0.8289473684210527
[Fold 3] accuracy : 0.8552631578947368
[Fold 4] accuracy : 0.8933333333333333
[Fold 5] accuracy : 0.8533333333333334
[Fold 6] accuracy : 0.8133333333333334
[Fold 7] accuracy : 0.84
[Fold 8] accuracy : 0.8533333333333334
[Fold 9] accuracy : 0.8933333333333333
Mean accuracy : 0.841529277739804


Min acc: 0.8415

## CatBoost: 0.9036

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score  # for printing score
# from sklearn.model_selection import GridSearchCV

In [ ]:
pip install catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
accs = []

model = CatBoostClassifier(random_state=RS, task_type='GPU', verbose=0)

for fold, (train_indice, test_indice) in enumerate(kfold.split(X, y)):
    X_train = X[train_indice]
    y_train = y[train_indice]
    X_test = X[test_indice]
    y_test = y[test_indice]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("[Fold {}] accuracy : {}".format(fold, accuracy_score(y_test, y_pred)))
    accs.append(acc)

print('Mean accuracy : {}'.format(np.mean(accs)))     

[Fold 0] accuracy : 0.8441558441558441
[Fold 1] accuracy : 0.8961038961038961
[Fold 2] accuracy : 0.8947368421052632
[Fold 3] accuracy : 0.8947368421052632
[Fold 4] accuracy : 0.9333333333333333
[Fold 5] accuracy : 0.9066666666666666
[Fold 6] accuracy : 0.8666666666666667
[Fold 7] accuracy : 0.92
[Fold 8] accuracy : 0.9466666666666667
[Fold 9] accuracy : 0.9333333333333333
Mean accuracy : 0.9036400091136934


그냥 해도 정확도 0.9167(RS=123), 0.9234(RS=456).
그런데 칼럼이 너무 많으니까 줄여보자

## Feature selection

### BorutaShap으로 가중치가 높은 피쳐를 찾자(LightGBM)

In [ ]:
pip install BorutaShap

In [ ]:
from BorutaShap import BorutaShap
from lightgbm import LGBMClassifier

df.head()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,female,male,id,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,rapJitter,ppq5Jitter,ddpJitter,locShimmer,locDbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer,meanAutoCorrHarmonicity,meanNoiseToHarmHarmonicity,meanHarmToNoiseHarmonicity,minIntensity,maxIntensity,meanIntensity,f1,f2,f3,f4,b1,b2,b3,b4,GQ_prc5_95,GQ_std_cycle_open,GQ_std_cycle_closed,GNE_mean,GNE_std,...,tqwt_skewnessValue_dec_34,tqwt_skewnessValue_dec_35,tqwt_skewnessValue_dec_36,tqwt_kurtosisValue_dec_1,tqwt_kurtosisValue_dec_2,tqwt_kurtosisValue_dec_3,tqwt_kurtosisValue_dec_4,tqwt_kurtosisValue_dec_5,tqwt_kurtosisValue_dec_6,tqwt_kurtosisValue_dec_7,tqwt_kurtosisValue_dec_8,tqwt_kurtosisValue_dec_9,tqwt_kurtosisValue_dec_10,tqwt_kurtosisValue_dec_11,tqwt_kurtosisValue_dec_12,tqwt_kurtosisValue_dec_13,tqwt_kurtosisValue_dec_14,tqwt_kurtosisValue_dec_15,tqwt_kurtosisValue_dec_16,tqwt_kurtosisValue_dec_17,tqwt_kurtosisValue_dec_18,tqwt_kurtosisValue_dec_19,tqwt_kurtosisValue_dec_20,tqwt_kurtosisValue_dec_21,tqwt_kurtosisValue_dec_22,tqwt_kurtosisValue_dec_23,tqwt_kurtosisValue_dec_24,tqwt_kurtosisValue_dec_25,tqwt_kurtosisValue_dec_26,tqwt_kurtosisValue_dec_27,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,0.000018,0.00067,0.00129,0.00200,0.05883,0.517,0.03011,0.03496,0.04828,0.09034,0.970805,0.036223,18.995,69.997496,76.088046,72.465512,539.342735,1031.849040,2447.162183,3655.054806,101.092218,83.147440,255.214830,396.643631,0.77778,11.7245,2.8277,1.17300,0.26512,...,0.071728,0.010352,-2.73030,66.5007,36.9934,26.3508,51.9577,21.5451,8.2488,8.0024,6.9635,6.9189,5.0622,4.6068,7.7218,2.7224,2.4171,2.9383,4.2077,3.1541,2.8531,2.7496,2.1550,2.9457,2.1993,1.9830,1.8314,2.0062,1.6058,1.5466,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,0.000016,0.00052,0.00112,0.00157,0.05516,0.502,0.02320,0.03675,0.06195,0.06961,0.984322,0.017974,21.497,67.415903,73.046374,71.528945,564.363614,1016.367294,2383.565201,3498.681572,58.465428,86.487292,248.357127,218.229722,0.81250,13.8284,2.8908,1.02210,0.22004,...,0.729330,0.780410,5.22940,8643.9860,3962.0554,2976.2411,4329.0607,4005.1329,1127.6762,116.5331,20.2332,13.6395,12.5514,13.6965,60.8974,6.2958,3.0323,3.4039,4.3794,2.4329,2.0585,2.1839,2.2061,3.0999,1.9824,1.6227,1.5783,2.0470,1.5772,1.5530,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,0.000015,0.00057,0.00111,0.00171,0.09902,0.897,0.05094,0.06497,0.07772,0.15282,0.974846,0.026313,17.651,62.661706,71.633549,68.086583,548.444604,1032.406341,2357.826954,3678.128717,160.387771,54.685168,151.694847,84.240339,0.81818,26.9273,2.6975,0.84951,0.15756,...,0.269510,-0.005522,0.35054,29.2717,32.4971,38.9453,85.1480,62.5132,9.7308,6.8890,6.1357,5.4926,3.9078,4.0864,6.9828,3.4411,3.5173,3.7204,4.2212,3.4881,3.4851,3.3007,2.0427,3.1436,2.1203,1.6627,1.6731,3.2597,1.5921,1.5399,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,1,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,0.000046,0.00149,0.00268,0.00446,0.05451,0.527,0.02395,0.02857,0.04462,0.07185,0.968343,0.042003,19.865,76.306989,81.000749,79.190593,819.529588,1201.813897,3154.035654,4122.163933,238.667052,191.984916,573.752909,526.147599,0.98548,139.5744,1.6961,0.83405,0.17295,...,0.366920,-0.492650,0.19164,591.0116,65.3827,53.9852,45.4458,34.6650,88.3259,46.6869,23.2911,19.2022,42.3957,100.3649,7.1967,6.3892,6.5496,6.0264,4.7656,4.8909,4.2531,3.0295,2.0362,1.8478,2.5776,2.2064,1.9491,1.9120,1.8829,6.9761,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,1,0.32790,0.79782,0.53028,236,235,0.008

In [ ]:
model = LGBMClassifier(random_state=RS, device='gpu')

# if classification is False it is a Regression problem
Feature_Selector = BorutaShap(model=model,
                              importance_measure='shap',
                              classification=True)

Feature_Selector.fit(X=df.iloc[:, :-1], y=df['class'],
                     n_trials=100, sample=False,
                     train_or_test = 'test', normalize=True,
                     verbose=True)

중요한 피쳐 36개:

['tqwt_entropy_log_dec_12', 'tqwt_minValue_dec_17', 'tqwt_energy_dec_12', 'DFA', 'std_MFCC_2nd_coef', 'mean_1st_delta_delta', 'std_6th_delta_delta', 'mean_MFCC_2nd_coef', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_17', 'tqwt_energy_dec_25', 'tqwt_TKEO_mean_dec_25', 'tqwt_TKEO_std_dec_25', 'tqwt_entropy_log_dec_27', 'tqwt_TKEO_std_dec_11', 'std_11th_delta_delta', 'tqwt_entropy_log_dec_33', 'IMF_NSR_TKEO', 'tqwt_skewnessValue_dec_25', 'std_delta_log_energy', 'tqwt_energy_dec_26', 'std_8th_delta_delta', 'std_7th_delta_delta', 'tqwt_TKEO_std_dec_20', 'tqwt_minValue_dec_13', 'tqwt_skewnessValue_dec_12', 'tqwt_energy_dec_6', 'mean_MFCC_1st_coef', 'tqwt_kurtosisValue_dec_20', 'mean_MFCC_6th_coef', 'tqwt_meanValue_dec_36', 'tqwt_entropy_log_dec_35', 'mean_2nd_delta', 'tqwt_stdValue_dec_18', 'tqwt_meanValue_dec_5', 'minIntensity']

불확실한 피쳐 13개:

['tqwt_skewnessValue_dec_36', 'tqwt_energy_dec_27', 'tqwt_kurtosisValue_dec_18', 'tqwt_TKEO_std_dec_19', 'tqwt_energy_dec_11', 'tqwt_energy_dec_7', 'std_6th_delta', 'mean_delta_delta_log_energy', 'tqwt_entropy_log_dec_16', 'tqwt_entropy_log_dec_32', 'tqwt_entropy_shannon_dec_25', 'tqwt_medianValue_dec_1', 'tqwt_kurtosisValue_dec_36']

나머지 712개는 안중요

#### BorutaShap 결과를 기반으로 LGBM돌리기(불확실 feature 없이): 0.9287

일단 10 fold부터 만들고 시작하자

In [ ]:
# StratifiedKFold guerantees uniform proportion of target values for each fold
from sklearn.model_selection import StratifiedKFold

RS = 456
# shuffle: random shuffle data, random_state: key of random shuffle
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=RS)

X = df[['tqwt_entropy_log_dec_12', 'tqwt_minValue_dec_17', 'tqwt_energy_dec_12', 'DFA', 'std_MFCC_2nd_coef', 'mean_1st_delta_delta', 'std_6th_delta_delta', 'mean_MFCC_2nd_coef', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_17', 'tqwt_energy_dec_25', 'tqwt_TKEO_mean_dec_25', 'tqwt_TKEO_std_dec_25', 'tqwt_entropy_log_dec_27', 'tqwt_TKEO_std_dec_11', 'std_11th_delta_delta', 'tqwt_entropy_log_dec_33', 'IMF_NSR_TKEO', 'tqwt_skewnessValue_dec_25', 'std_delta_log_energy', 'tqwt_energy_dec_26', 'std_8th_delta_delta', 'std_7th_delta_delta', 'tqwt_TKEO_std_dec_20', 'tqwt_minValue_dec_13', 'tqwt_skewnessValue_dec_12', 'tqwt_energy_dec_6', 'mean_MFCC_1st_coef', 'tqwt_kurtosisValue_dec_20', 'mean_MFCC_6th_coef', 'tqwt_meanValue_dec_36', 'tqwt_entropy_log_dec_35', 'mean_2nd_delta', 'tqwt_stdValue_dec_18', 'tqwt_meanValue_dec_5', 'minIntensity']].values
y = df['class'].values

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score  # for printing score

from lightgbm import LGBMClassifier

In [ ]:
accs = []

model = LGBMClassifier(random_state=RS, device='gpu')

for fold, (train_indice, test_indice) in enumerate(kfold.split(X, y)):
    X_train = X[train_indice]
    y_train = y[train_indice]
    X_test = X[test_indice]
    y_test = y[test_indice]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("[Fold {}] accuracy : {}".format(fold, accuracy_score(y_test, y_pred)))
    accs.append(acc)

print('Mean accuracy : {}'.format(np.mean(accs)))     

[Fold 0] accuracy : 0.8701298701298701
[Fold 1] accuracy : 0.935064935064935
[Fold 2] accuracy : 0.9078947368421053
[Fold 3] accuracy : 0.9473684210526315
[Fold 4] accuracy : 0.92
[Fold 5] accuracy : 0.92
[Fold 6] accuracy : 0.8933333333333333
[Fold 7] accuracy : 0.9466666666666667
[Fold 8] accuracy : 0.9733333333333334
[Fold 9] accuracy : 0.9733333333333334
Mean accuracy : 0.9287124629756207


#### BorutaShap 결과를 기반으로 LGBM돌리기(불확실 feature 포함): 0.9353(채택)

In [ ]:
# StratifiedKFold guerantees uniform proportion of target values for each fold
from sklearn.model_selection import StratifiedKFold

RS = 456
# shuffle: random shuffle data, random_state: key of random shuffle
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=RS)

X = df[['tqwt_entropy_log_dec_12', 'tqwt_minValue_dec_17', 'tqwt_energy_dec_12', 'DFA', 'std_MFCC_2nd_coef', 'mean_1st_delta_delta', 'std_6th_delta_delta', 'mean_MFCC_2nd_coef', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_17', 'tqwt_energy_dec_25', 'tqwt_TKEO_mean_dec_25', 'tqwt_TKEO_std_dec_25', 'tqwt_entropy_log_dec_27', 'tqwt_TKEO_std_dec_11', 'std_11th_delta_delta', 'tqwt_entropy_log_dec_33', 'IMF_NSR_TKEO', 'tqwt_skewnessValue_dec_25', 'std_delta_log_energy', 'tqwt_energy_dec_26', 'std_8th_delta_delta', 'std_7th_delta_delta', 'tqwt_TKEO_std_dec_20', 'tqwt_minValue_dec_13', 'tqwt_skewnessValue_dec_12', 'tqwt_energy_dec_6', 'mean_MFCC_1st_coef', 'tqwt_kurtosisValue_dec_20', 'mean_MFCC_6th_coef', 'tqwt_meanValue_dec_36', 'tqwt_entropy_log_dec_35', 'mean_2nd_delta', 'tqwt_stdValue_dec_18', 'tqwt_meanValue_dec_5', 'minIntensity', 'tqwt_skewnessValue_dec_36', 'tqwt_energy_dec_27', 'tqwt_kurtosisValue_dec_18', 'tqwt_TKEO_std_dec_19', 'tqwt_energy_dec_11', 'tqwt_energy_dec_7', 'std_6th_delta', 'mean_delta_delta_log_energy', 'tqwt_entropy_log_dec_16', 'tqwt_entropy_log_dec_32', 'tqwt_entropy_shannon_dec_25', 'tqwt_medianValue_dec_1', 'tqwt_kurtosisValue_dec_36']].values
y = df['class'].values

import numpy as np
from sklearn.metrics import accuracy_score  # for printing score

from lightgbm import LGBMClassifier

In [ ]:
accs = []

model = LGBMClassifier(random_state=RS, device='gpu')

for fold, (train_indice, test_indice) in enumerate(kfold.split(X, y)):
    X_train = X[train_indice]
    y_train = y[train_indice]
    X_test = X[test_indice]
    y_test = y[test_indice]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("[Fold {}] accuracy : {}".format(fold, accuracy_score(y_test, y_pred)))
    accs.append(acc)

print('Mean accuracy : {}'.format(np.mean(accs)))

[Fold 0] accuracy : 0.8571428571428571
[Fold 1] accuracy : 0.948051948051948
[Fold 2] accuracy : 0.9342105263157895
[Fold 3] accuracy : 0.9473684210526315
[Fold 4] accuracy : 0.9466666666666667
[Fold 5] accuracy : 0.9333333333333333
[Fold 6] accuracy : 0.9066666666666666
[Fold 7] accuracy : 0.9466666666666667
[Fold 8] accuracy : 0.9466666666666667
[Fold 9] accuracy : 0.9866666666666667
Mean accuracy : 0.9353440419229893


#### LGBM gpu 세팅

In [ ]:
!git clone https://github.com/Microsoft/LightGBM

Cloning into 'LightGBM'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 20667 (delta 6), reused 4 (delta 1), pack-reused 20646
Receiving objects: 100% (20667/20667), 16.06 MiB | 30.46 MiB/s, done.
Resolving deltas: 100% (15083/15083), done.


In [ ]:
cd LightGBM

/content/LightGBM


In [ ]:
!mkdir build

In [ ]:
!cmake -DUSE_GPU=1

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Looking for CL_VERSION_2_2
-- Looking for CL_VERSION_2_2 - found
-- Found OpenCL: /usr/lib/x86_64-linux-gnu/libOpenCL.so (found version "2.2") 
-- OpenCL include directory: /usr/include
-- Boost version: 1.65.1
-- Found the following Boost libraries:
--

In [ ]:
!make -j$(nproc)

Scanning dependencies of target _lightgbm
Scanning dependencies of target lightgbm
[  1%] Building CXX object CMakeFiles/_lightgbm.dir/src/boosting/boosting.cpp.o
[  2%] Building CXX object CMakeFiles/lightgbm.dir/src/main.cpp.o
[  4%] Building CXX object CMakeFiles/lightgbm.dir/src/application/application.cpp.o
[  5%] Building CXX object CMakeFiles/lightgbm.dir/src/boosting/boosting.cpp.o
[  7%] Building CXX object CMakeFiles/_lightgbm.dir/src/boosting/gbdt.cpp.o
[  8%] Building CXX object CMakeFiles/_lightgbm.dir/src/boosting/gbdt_model_text.cpp.o
[ 10%] Building CXX object CMakeFiles/lightgbm.dir/src/boosting/gbdt.cpp.o
[ 11%] Building CXX object CMakeFiles/_lightgbm.dir/src/boosting/gbdt_prediction.cpp.o
[ 13%] Building CXX object CMakeFiles/lightgbm.dir/src/boosting/gbdt_model_text.cpp.o
[ 14%] Building CXX object CMakeFiles/_lightgbm.dir/src/boosting/prediction_early_stop.cpp.o
[ 16%] Building CXX object CMakeFiles/_lightgbm.dir/src/io/bin.cpp.o
[ 17%] Building CXX object CMakeFi

In [ ]:
!sudo apt-get -y install python-pip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpython-all-dev python-all python-all-dev python-asn1crypto
  python-cffi-backend python-crypto python-cryptography python-dbus
  python-enum34 python-gi python-idna python-ipaddress python-keyring
  python-keyrings.alt python-pip-whl python-pkg-resources python-secretstorage
  python-setuptools python-six python-wheel python-xdg
Suggested packages:
  python-crypto-doc python-cryptography-doc python-cryptography-vectors
  python-dbus-dbg python-dbus-doc python-enum34-doc python-gi-cairo
  gnome-keyring libkf5wallet-bin gir1.2-gnomekeyring-1.0 python-fs
  python-gdata python-keyczar python-secretstorage-doc python-setuptools-doc
The following NEW packages will be installed:
  libpython-all-dev python-all python-all-dev python-asn1crypto
  python-cffi-backend python-crypto python-cryptography python-dbus
  python-enum34 python-gi python-

In [ ]:
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (50.3.2)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.1.4)
     |████████████████████████████████| 14.5MB 211kB/s 
     |████████████████████████████████| 25.9MB 127kB/s 
     |████████████████████████████████| 6.8MB 54.2MB/s 
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.5.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Suc

In [ ]:
%cd /content/LightGBM/python-package

/content/LightGBM/python-package


In [ ]:
!sudo python setup.py install

running install
INFO:LightGBM:Starting to compile the library.
INFO:LightGBM:Starting to compile with CMake.
running build
running build_py
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
creating build
creating build/lib
creating build/lib/lightgbm
copying lightgbm/compat.py -> build/lib/lightgbm
copying lightgbm/sklearn.py -> build/lib/lightgbm
copying lightgbm/libpath.py -> build/lib/lightgbm
copying lightgbm/basic.py -> build/lib/lightgbm
copying lightgbm/plotting.py -> build/lib/lightgbm
copying lightgbm/__init__.py -> build/lib/lightgbm
copying lightgbm/engine.py -> build/lib/lightgbm
copying lightgbm/callback.py -> build/lib/lightgbm
running egg_info
creating lightgbm.egg-info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to ligh

In [ ]:
!pip3 uninstall scikit-learn

Uninstalling scikit-learn-0.23.2:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/scikit_learn-0.23.2.dist-info/*
    /usr/local/lib/python3.6/dist-packages/scikit_learn.libs/libgomp-3300acd3.so.1.0.0
    /usr/local/lib/python3.6/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-0.23.2


In [ ]:
!pip3 install scikit-learn==0.21.3

     |████████████████████████████████| 6.7MB 12.4MB/s 


In [ ]:
pip list | grep scikit-learn # 사이킷런 버전확인

scikit-learn                  0.21.3         


#### 하이퍼파라미터 튜닝

##### LGBM 그리드서치

In [ ]:
X = df[['tqwt_entropy_log_dec_12', 'tqwt_minValue_dec_17', 'tqwt_energy_dec_12', 'DFA', 'std_MFCC_2nd_coef', 'mean_1st_delta_delta', 'std_6th_delta_delta', 'mean_MFCC_2nd_coef', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_17', 'tqwt_energy_dec_25', 'tqwt_TKEO_mean_dec_25', 'tqwt_TKEO_std_dec_25', 'tqwt_entropy_log_dec_27', 'tqwt_TKEO_std_dec_11', 'std_11th_delta_delta', 'tqwt_entropy_log_dec_33', 'IMF_NSR_TKEO', 'tqwt_skewnessValue_dec_25', 'std_delta_log_energy', 'tqwt_energy_dec_26', 'std_8th_delta_delta', 'std_7th_delta_delta', 'tqwt_TKEO_std_dec_20', 'tqwt_minValue_dec_13', 'tqwt_skewnessValue_dec_12', 'tqwt_energy_dec_6', 'mean_MFCC_1st_coef', 'tqwt_kurtosisValue_dec_20', 'mean_MFCC_6th_coef', 'tqwt_meanValue_dec_36', 'tqwt_entropy_log_dec_35', 'mean_2nd_delta', 'tqwt_stdValue_dec_18', 'tqwt_meanValue_dec_5', 'minIntensity', 'tqwt_skewnessValue_dec_36', 'tqwt_energy_dec_27', 'tqwt_kurtosisValue_dec_18', 'tqwt_TKEO_std_dec_19', 'tqwt_energy_dec_11', 'tqwt_energy_dec_7', 'std_6th_delta', 'mean_delta_delta_log_energy', 'tqwt_entropy_log_dec_16', 'tqwt_entropy_log_dec_32', 'tqwt_entropy_shannon_dec_25', 'tqwt_medianValue_dec_1', 'tqwt_kurtosisValue_dec_36']].values
y = df['class'].values

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
model = LGBMClassifier(random_state=RS, device='gpu')

param_grid = dict(learning_rate=[0.07, 0.08, 0.09, 0.1], # default=0.1
                  # boosting learning rate
                  num_leaves=[32, 128, 512], # default=31
                  n_estimators=[50, 100, 200, 300], # default=100
                  max_depth=[5, 7, 9, None] # default=None
                  )

gs = GridSearchCV(estimator=model,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=10,
                  n_jobs=1) # This should be 1 to use GPU

In [ ]:
gs.fit(X, y)
print(gs.best_score_)
print(gs.best_params_)

0.9007936507936508
{'learning_rate': 0.09, 'max_depth': None, 'n_estimators': 200, 'num_leaves': 32}


결과:{'learning_rate': 0.09, 'max_depth': None, 'n_estimators': 200, 'num_leaves': 32}, acc=0.9008

쏠리지 않은 n_estimators와 learning rate와 더 줄일 수 없는 num_leaves는 그대로 입력하고 다시 돌려보자

In [ ]:
model = LGBMClassifier(random_state=RS, device='gpu', n_estimators=200,
                       learning_rate=0.09, num_leaves=32)

param_grid = dict(max_depth=[13, 15, 18, None]) 
# 이전결과: default=None

gs = GridSearchCV(estimator=model,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=10,
                  n_jobs=1) # This should be 1 to use GPU

In [ ]:
gs.fit(X, y)
print(gs.best_score_)
print(gs.best_params_)

0.9007936507936508
{'max_depth': 18}


결과: {'learning_rate': 0.09, 'max_depth': 18, 'n_estimators': 200, 'num_leaves': 32}, acc=0.9008

정확도상 소수점 끝까지 가도 차이가 없다. 걍 {'learning_rate': 0.09, 'n_estimators': 200, 'num_leaves': 32} 정도로 쓰면 되겠다.

##### 튜닝값으로 BorutaShap 결과를 기반으로 LGBM돌리기(불확실 feature 포함): 0.9407

일단 10 fold부터 만들고 시작하자

In [ ]:
# StratifiedKFold guerantees uniform proportion of target values for each fold
from sklearn.model_selection import StratifiedKFold

RS = 456
# shuffle: random shuffle data, random_state: key of random shuffle
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=RS)

X = df[['tqwt_entropy_log_dec_12', 'tqwt_minValue_dec_17', 'tqwt_energy_dec_12', 'DFA', 'std_MFCC_2nd_coef', 'mean_1st_delta_delta', 'std_6th_delta_delta', 'mean_MFCC_2nd_coef', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_17', 'tqwt_energy_dec_25', 'tqwt_TKEO_mean_dec_25', 'tqwt_TKEO_std_dec_25', 'tqwt_entropy_log_dec_27', 'tqwt_TKEO_std_dec_11', 'std_11th_delta_delta', 'tqwt_entropy_log_dec_33', 'IMF_NSR_TKEO', 'tqwt_skewnessValue_dec_25', 'std_delta_log_energy', 'tqwt_energy_dec_26', 'std_8th_delta_delta', 'std_7th_delta_delta', 'tqwt_TKEO_std_dec_20', 'tqwt_minValue_dec_13', 'tqwt_skewnessValue_dec_12', 'tqwt_energy_dec_6', 'mean_MFCC_1st_coef', 'tqwt_kurtosisValue_dec_20', 'mean_MFCC_6th_coef', 'tqwt_meanValue_dec_36', 'tqwt_entropy_log_dec_35', 'mean_2nd_delta', 'tqwt_stdValue_dec_18', 'tqwt_meanValue_dec_5', 'minIntensity', 'tqwt_skewnessValue_dec_36', 'tqwt_energy_dec_27', 'tqwt_kurtosisValue_dec_18', 'tqwt_TKEO_std_dec_19', 'tqwt_energy_dec_11', 'tqwt_energy_dec_7', 'std_6th_delta', 'mean_delta_delta_log_energy', 'tqwt_entropy_log_dec_16', 'tqwt_entropy_log_dec_32', 'tqwt_entropy_shannon_dec_25', 'tqwt_medianValue_dec_1', 'tqwt_kurtosisValue_dec_36']].values
y = df['class'].values

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score  # for printing score

from lightgbm import LGBMClassifier

In [ ]:
accs = []

model = LGBMClassifier(random_state=RS, device='gpu',
                       learning_rate=0.09,
                       n_estimators=200, num_leaves=32)

for fold, (train_indice, test_indice) in enumerate(kfold.split(X, y)):
    X_train = X[train_indice]
    y_train = y[train_indice]
    X_test = X[test_indice]
    y_test = y[test_indice]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("[Fold {}] accuracy : {}".format(fold, accuracy_score(y_test, y_pred)))
    accs.append(acc)

print('Mean accuracy : {}'.format(np.mean(accs)))     

[Fold 0] accuracy : 0.8831168831168831
[Fold 1] accuracy : 0.935064935064935
[Fold 2] accuracy : 0.9342105263157895
[Fold 3] accuracy : 0.9342105263157895
[Fold 4] accuracy : 0.9333333333333333
[Fold 5] accuracy : 0.9466666666666667
[Fold 6] accuracy : 0.9066666666666666
[Fold 7] accuracy : 0.9733333333333334
[Fold 8] accuracy : 0.9733333333333334
[Fold 9] accuracy : 0.9866666666666667
Mean accuracy : 0.9406602870813396


결과: 0.9407

In [ ]:
accs = []

model = LGBMClassifier(random_state=RS, device='gpu',
                       learning_rate=0.07, max_depth=9,
                       n_estimators=200, num_leaves=512,
                       subsample=0.4)

for fold, (train_indice, test_indice) in enumerate(kfold.split(X, y)):
    X_train = X[train_indice]
    y_train = y[train_indice]
    X_test = X[test_indice]
    y_test = y[test_indice]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("[Fold {}] accuracy : {}".format(fold, accuracy_score(y_test, y_pred)))
    accs.append(acc)

print('Mean accuracy : {}'.format(np.mean(accs)))     

[Fold 0] accuracy : 0.935064935064935
[Fold 1] accuracy : 0.9090909090909091
[Fold 2] accuracy : 0.9342105263157895
[Fold 3] accuracy : 0.9736842105263158
[Fold 4] accuracy : 0.9333333333333333
[Fold 5] accuracy : 0.8666666666666667
[Fold 6] accuracy : 0.96
[Fold 7] accuracy : 0.9333333333333333
[Fold 8] accuracy : 0.9066666666666666
[Fold 9] accuracy : 0.9466666666666667
Mean accuracy : 0.9298717247664616


결과: 0.9299

subsample만 바뀌었는데 뭘로 하든지 별로 차이는 없는 것 같다. 그냥 기본모델 쓰는 게 나을듯

### BorutaShap으로 가중치가 높은 피쳐를 찾자(Catboost)

#### BorutaShap 구축

In [ ]:
!pip install BorutaShap

In [ ]:
!pip install catboost

In [ ]:
!pip install scikit-learn

In [ ]:
from BorutaShap import BorutaShap
from catboost import CatBoostClassifier

In [ ]:
df = df.drop(columns=['id'])

In [ ]:
RS = 456

model = CatBoostClassifier(random_state=RS, task_type='GPU')

# if classification is False it is a Regression problem
Feature_Selector = BorutaShap(model=model,
                              importance_measure='shap',
                              classification=True)

Feature_Selector.fit(X=df.iloc[:, :-1], y=df['class'],
                     n_trials=100, sample=False,
                     train_or_test = 'test', normalize=True,
                     verbose=True)

 33%|███▎      | 33/100 [31:03<36:56, 33.08s/it]

중요한 피쳐 36개:

['tqwt_entropy_log_dec_12', 'tqwt_minValue_dec_17', 'tqwt_energy_dec_12', 'DFA', 'std_MFCC_2nd_coef', 'mean_1st_delta_delta', 'std_6th_delta_delta', 'mean_MFCC_2nd_coef', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_17', 'tqwt_energy_dec_25', 'tqwt_TKEO_mean_dec_25', 'tqwt_TKEO_std_dec_25', 'tqwt_entropy_log_dec_27', 'tqwt_TKEO_std_dec_11', 'std_11th_delta_delta', 'tqwt_entropy_log_dec_33', 'IMF_NSR_TKEO', 'tqwt_skewnessValue_dec_25', 'std_delta_log_energy', 'tqwt_energy_dec_26', 'std_8th_delta_delta', 'std_7th_delta_delta', 'tqwt_TKEO_std_dec_20', 'tqwt_minValue_dec_13', 'tqwt_skewnessValue_dec_12', 'tqwt_energy_dec_6', 'mean_MFCC_1st_coef', 'tqwt_kurtosisValue_dec_20', 'mean_MFCC_6th_coef', 'tqwt_meanValue_dec_36', 'tqwt_entropy_log_dec_35', 'mean_2nd_delta', 'tqwt_stdValue_dec_18', 'tqwt_meanValue_dec_5', 'minIntensity']

불확실한 피쳐 13개:

['tqwt_skewnessValue_dec_36', 'tqwt_energy_dec_27', 'tqwt_kurtosisValue_dec_18', 'tqwt_TKEO_std_dec_19', 'tqwt_energy_dec_11', 'tqwt_energy_dec_7', 'std_6th_delta', 'mean_delta_delta_log_energy', 'tqwt_entropy_log_dec_16', 'tqwt_entropy_log_dec_32', 'tqwt_entropy_shannon_dec_25', 'tqwt_medianValue_dec_1', 'tqwt_kurtosisValue_dec_36']

나머지 712개는 안중요

#### BorutaShap 결과를 기반으로 Catboost 돌리기(불확실 feature 없이): 0.9287

일단 10 fold부터 만들고 시작하자

In [ ]:
# StratifiedKFold guerantees uniform proportion of target values for each fold
from sklearn.model_selection import StratifiedKFold

RS = 456
# shuffle: random shuffle data, random_state: key of random shuffle
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=RS)

X = df[['tqwt_entropy_log_dec_12', 'tqwt_minValue_dec_17', 'tqwt_energy_dec_12', 'DFA', 'std_MFCC_2nd_coef', 'mean_1st_delta_delta', 'std_6th_delta_delta', 'mean_MFCC_2nd_coef', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_17', 'tqwt_energy_dec_25', 'tqwt_TKEO_mean_dec_25', 'tqwt_TKEO_std_dec_25', 'tqwt_entropy_log_dec_27', 'tqwt_TKEO_std_dec_11', 'std_11th_delta_delta', 'tqwt_entropy_log_dec_33', 'IMF_NSR_TKEO', 'tqwt_skewnessValue_dec_25', 'std_delta_log_energy', 'tqwt_energy_dec_26', 'std_8th_delta_delta', 'std_7th_delta_delta', 'tqwt_TKEO_std_dec_20', 'tqwt_minValue_dec_13', 'tqwt_skewnessValue_dec_12', 'tqwt_energy_dec_6', 'mean_MFCC_1st_coef', 'tqwt_kurtosisValue_dec_20', 'mean_MFCC_6th_coef', 'tqwt_meanValue_dec_36', 'tqwt_entropy_log_dec_35', 'mean_2nd_delta', 'tqwt_stdValue_dec_18', 'tqwt_meanValue_dec_5', 'minIntensity']].values
y = df['class'].values

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score  # for printing score

from lightgbm import LGBMClassifier

In [ ]:
accs = []

model = LGBMClassifier(random_state=RS, device='gpu')

for fold, (train_indice, test_indice) in enumerate(kfold.split(X, y)):
    X_train = X[train_indice]
    y_train = y[train_indice]
    X_test = X[test_indice]
    y_test = y[test_indice]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("[Fold {}] accuracy : {}".format(fold, accuracy_score(y_test, y_pred)))
    accs.append(acc)

print('Mean accuracy : {}'.format(np.mean(accs)))     

[Fold 0] accuracy : 0.8701298701298701
[Fold 1] accuracy : 0.935064935064935
[Fold 2] accuracy : 0.9078947368421053
[Fold 3] accuracy : 0.9473684210526315
[Fold 4] accuracy : 0.92
[Fold 5] accuracy : 0.92
[Fold 6] accuracy : 0.8933333333333333
[Fold 7] accuracy : 0.9466666666666667
[Fold 8] accuracy : 0.9733333333333334
[Fold 9] accuracy : 0.9733333333333334
Mean accuracy : 0.9287124629756207


#### BorutaShap 결과를 기반으로 Catboost 돌리기(불확실 feature 포함): 0.9353(채택)

In [ ]:
# StratifiedKFold guerantees uniform proportion of target values for each fold
from sklearn.model_selection import StratifiedKFold

RS = 456
# shuffle: random shuffle data, random_state: key of random shuffle
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=RS)

X = df[['tqwt_entropy_log_dec_12', 'tqwt_minValue_dec_17', 'tqwt_energy_dec_12', 'DFA', 'std_MFCC_2nd_coef', 'mean_1st_delta_delta', 'std_6th_delta_delta', 'mean_MFCC_2nd_coef', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_17', 'tqwt_energy_dec_25', 'tqwt_TKEO_mean_dec_25', 'tqwt_TKEO_std_dec_25', 'tqwt_entropy_log_dec_27', 'tqwt_TKEO_std_dec_11', 'std_11th_delta_delta', 'tqwt_entropy_log_dec_33', 'IMF_NSR_TKEO', 'tqwt_skewnessValue_dec_25', 'std_delta_log_energy', 'tqwt_energy_dec_26', 'std_8th_delta_delta', 'std_7th_delta_delta', 'tqwt_TKEO_std_dec_20', 'tqwt_minValue_dec_13', 'tqwt_skewnessValue_dec_12', 'tqwt_energy_dec_6', 'mean_MFCC_1st_coef', 'tqwt_kurtosisValue_dec_20', 'mean_MFCC_6th_coef', 'tqwt_meanValue_dec_36', 'tqwt_entropy_log_dec_35', 'mean_2nd_delta', 'tqwt_stdValue_dec_18', 'tqwt_meanValue_dec_5', 'minIntensity', 'tqwt_skewnessValue_dec_36', 'tqwt_energy_dec_27', 'tqwt_kurtosisValue_dec_18', 'tqwt_TKEO_std_dec_19', 'tqwt_energy_dec_11', 'tqwt_energy_dec_7', 'std_6th_delta', 'mean_delta_delta_log_energy', 'tqwt_entropy_log_dec_16', 'tqwt_entropy_log_dec_32', 'tqwt_entropy_shannon_dec_25', 'tqwt_medianValue_dec_1', 'tqwt_kurtosisValue_dec_36']].values
y = df['class'].values

import numpy as np
from sklearn.metrics import accuracy_score  # for printing score

from lightgbm import LGBMClassifier

In [ ]:
accs = []

model = LGBMClassifier(random_state=RS, device='gpu')

for fold, (train_indice, test_indice) in enumerate(kfold.split(X, y)):
    X_train = X[train_indice]
    y_train = y[train_indice]
    X_test = X[test_indice]
    y_test = y[test_indice]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("[Fold {}] accuracy : {}".format(fold, accuracy_score(y_test, y_pred)))
    accs.append(acc)

print('Mean accuracy : {}'.format(np.mean(accs)))

[Fold 0] accuracy : 0.8571428571428571
[Fold 1] accuracy : 0.948051948051948
[Fold 2] accuracy : 0.9342105263157895
[Fold 3] accuracy : 0.9473684210526315
[Fold 4] accuracy : 0.9466666666666667
[Fold 5] accuracy : 0.9333333333333333
[Fold 6] accuracy : 0.9066666666666666
[Fold 7] accuracy : 0.9466666666666667
[Fold 8] accuracy : 0.9466666666666667
[Fold 9] accuracy : 0.9866666666666667
Mean accuracy : 0.9353440419229893


#### 하이퍼파라미터 튜닝

##### LGBM 그리드서치

In [ ]:
X = df[['tqwt_entropy_log_dec_12', 'tqwt_minValue_dec_17', 'tqwt_energy_dec_12', 'DFA', 'std_MFCC_2nd_coef', 'mean_1st_delta_delta', 'std_6th_delta_delta', 'mean_MFCC_2nd_coef', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_17', 'tqwt_energy_dec_25', 'tqwt_TKEO_mean_dec_25', 'tqwt_TKEO_std_dec_25', 'tqwt_entropy_log_dec_27', 'tqwt_TKEO_std_dec_11', 'std_11th_delta_delta', 'tqwt_entropy_log_dec_33', 'IMF_NSR_TKEO', 'tqwt_skewnessValue_dec_25', 'std_delta_log_energy', 'tqwt_energy_dec_26', 'std_8th_delta_delta', 'std_7th_delta_delta', 'tqwt_TKEO_std_dec_20', 'tqwt_minValue_dec_13', 'tqwt_skewnessValue_dec_12', 'tqwt_energy_dec_6', 'mean_MFCC_1st_coef', 'tqwt_kurtosisValue_dec_20', 'mean_MFCC_6th_coef', 'tqwt_meanValue_dec_36', 'tqwt_entropy_log_dec_35', 'mean_2nd_delta', 'tqwt_stdValue_dec_18', 'tqwt_meanValue_dec_5', 'minIntensity', 'tqwt_skewnessValue_dec_36', 'tqwt_energy_dec_27', 'tqwt_kurtosisValue_dec_18', 'tqwt_TKEO_std_dec_19', 'tqwt_energy_dec_11', 'tqwt_energy_dec_7', 'std_6th_delta', 'mean_delta_delta_log_energy', 'tqwt_entropy_log_dec_16', 'tqwt_entropy_log_dec_32', 'tqwt_entropy_shannon_dec_25', 'tqwt_medianValue_dec_1', 'tqwt_kurtosisValue_dec_36']].values
y = df['class'].values

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
model = LGBMClassifier(random_state=RS, device='gpu')

param_grid = dict(learning_rate=[0.07, 0.08, 0.09, 0.1], # default=0.1
                  # boosting learning rate
                  num_leaves=[32, 128, 512], # default=31
                  n_estimators=[50, 100, 200, 300], # default=100
                  max_depth=[5, 7, 9, None] # default=None
                  )

gs = GridSearchCV(estimator=model,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=10,
                  n_jobs=1) # This should be 1 to use GPU

In [ ]:
gs.fit(X, y)
print(gs.best_score_)
print(gs.best_params_)

0.9007936507936508
{'learning_rate': 0.09, 'max_depth': None, 'n_estimators': 200, 'num_leaves': 32}


결과:{'learning_rate': 0.09, 'max_depth': None, 'n_estimators': 200, 'num_leaves': 32}, acc=0.9008

쏠리지 않은 n_estimators와 learning rate와 더 줄일 수 없는 num_leaves는 그대로 입력하고 다시 돌려보자

In [ ]:
model = LGBMClassifier(random_state=RS, device='gpu', n_estimators=200,
                       learning_rate=0.09, num_leaves=32)

param_grid = dict(max_depth=[13, 15, 18, None]) 
# 이전결과: default=None

gs = GridSearchCV(estimator=model,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=10,
                  n_jobs=1) # This should be 1 to use GPU

In [ ]:
gs.fit(X, y)
print(gs.best_score_)
print(gs.best_params_)

0.9007936507936508
{'max_depth': 18}


결과: {'learning_rate': 0.09, 'max_depth': 18, 'n_estimators': 200, 'num_leaves': 32}, acc=0.9008

정확도상 소수점 끝까지 가도 차이가 없다. 걍 {'learning_rate': 0.09, 'n_estimators': 200, 'num_leaves': 32} 정도로 쓰면 되겠다.

##### 튜닝값으로 BorutaShap 결과를 기반으로 LGBM돌리기(불확실 feature 포함): 0.9407

일단 10 fold부터 만들고 시작하자

In [ ]:
# StratifiedKFold guerantees uniform proportion of target values for each fold
from sklearn.model_selection import StratifiedKFold

RS = 456
# shuffle: random shuffle data, random_state: key of random shuffle
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=RS)

X = df[['tqwt_entropy_log_dec_12', 'tqwt_minValue_dec_17', 'tqwt_energy_dec_12', 'DFA', 'std_MFCC_2nd_coef', 'mean_1st_delta_delta', 'std_6th_delta_delta', 'mean_MFCC_2nd_coef', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_17', 'tqwt_energy_dec_25', 'tqwt_TKEO_mean_dec_25', 'tqwt_TKEO_std_dec_25', 'tqwt_entropy_log_dec_27', 'tqwt_TKEO_std_dec_11', 'std_11th_delta_delta', 'tqwt_entropy_log_dec_33', 'IMF_NSR_TKEO', 'tqwt_skewnessValue_dec_25', 'std_delta_log_energy', 'tqwt_energy_dec_26', 'std_8th_delta_delta', 'std_7th_delta_delta', 'tqwt_TKEO_std_dec_20', 'tqwt_minValue_dec_13', 'tqwt_skewnessValue_dec_12', 'tqwt_energy_dec_6', 'mean_MFCC_1st_coef', 'tqwt_kurtosisValue_dec_20', 'mean_MFCC_6th_coef', 'tqwt_meanValue_dec_36', 'tqwt_entropy_log_dec_35', 'mean_2nd_delta', 'tqwt_stdValue_dec_18', 'tqwt_meanValue_dec_5', 'minIntensity', 'tqwt_skewnessValue_dec_36', 'tqwt_energy_dec_27', 'tqwt_kurtosisValue_dec_18', 'tqwt_TKEO_std_dec_19', 'tqwt_energy_dec_11', 'tqwt_energy_dec_7', 'std_6th_delta', 'mean_delta_delta_log_energy', 'tqwt_entropy_log_dec_16', 'tqwt_entropy_log_dec_32', 'tqwt_entropy_shannon_dec_25', 'tqwt_medianValue_dec_1', 'tqwt_kurtosisValue_dec_36']].values
y = df['class'].values

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score  # for printing score

from lightgbm import LGBMClassifier

In [ ]:
accs = []

model = LGBMClassifier(random_state=RS, device='gpu',
                       learning_rate=0.09,
                       n_estimators=200, num_leaves=32)

for fold, (train_indice, test_indice) in enumerate(kfold.split(X, y)):
    X_train = X[train_indice]
    y_train = y[train_indice]
    X_test = X[test_indice]
    y_test = y[test_indice]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("[Fold {}] accuracy : {}".format(fold, accuracy_score(y_test, y_pred)))
    accs.append(acc)

print('Mean accuracy : {}'.format(np.mean(accs)))     

[Fold 0] accuracy : 0.8831168831168831
[Fold 1] accuracy : 0.935064935064935
[Fold 2] accuracy : 0.9342105263157895
[Fold 3] accuracy : 0.9342105263157895
[Fold 4] accuracy : 0.9333333333333333
[Fold 5] accuracy : 0.9466666666666667
[Fold 6] accuracy : 0.9066666666666666
[Fold 7] accuracy : 0.9733333333333334
[Fold 8] accuracy : 0.9733333333333334
[Fold 9] accuracy : 0.9866666666666667
Mean accuracy : 0.9406602870813396


결과: 0.9407

In [ ]:
accs = []

model = LGBMClassifier(random_state=RS, device='gpu',
                       learning_rate=0.07, max_depth=9,
                       n_estimators=200, num_leaves=512,
                       subsample=0.4)

for fold, (train_indice, test_indice) in enumerate(kfold.split(X, y)):
    X_train = X[train_indice]
    y_train = y[train_indice]
    X_test = X[test_indice]
    y_test = y[test_indice]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("[Fold {}] accuracy : {}".format(fold, accuracy_score(y_test, y_pred)))
    accs.append(acc)

print('Mean accuracy : {}'.format(np.mean(accs)))     

[Fold 0] accuracy : 0.935064935064935
[Fold 1] accuracy : 0.9090909090909091
[Fold 2] accuracy : 0.9342105263157895
[Fold 3] accuracy : 0.9736842105263158
[Fold 4] accuracy : 0.9333333333333333
[Fold 5] accuracy : 0.8666666666666667
[Fold 6] accuracy : 0.96
[Fold 7] accuracy : 0.9333333333333333
[Fold 8] accuracy : 0.9066666666666666
[Fold 9] accuracy : 0.9466666666666667
Mean accuracy : 0.9298717247664616


결과: 0.9299

subsample만 바뀌었는데 뭘로 하든지 별로 차이는 없는 것 같다. 그냥 기본모델 쓰는 게 나을듯

# ID 중복제거 후 AUC 비교
같은 ID를 가진 데이터가 train과 test에 모두 들어가면 overfit의 위험이 있기 때문에, 이를 방지한다.

그리고 데이터가 inbalance한 상황에서 accuracy는 여러 모델의 성능을 비교하기에 부적합하므로 AUC를 대신 도입한다.

## ID 중복방지

###  Gender를 category로 인식시키기 위해 one-hot encoding

In [ ]:
df = speech.copy()
df

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,rapJitter,ppq5Jitter,ddpJitter,locShimmer,locDbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer,meanAutoCorrHarmonicity,meanNoiseToHarmHarmonicity,meanHarmToNoiseHarmonicity,minIntensity,maxIntensity,meanIntensity,f1,f2,f3,f4,b1,b2,b3,b4,GQ_prc5_95,GQ_std_cycle_open,GQ_std_cycle_closed,GNE_mean,GNE_std,GNE_SNR_TKEO,...,tqwt_skewnessValue_dec_34,tqwt_skewnessValue_dec_35,tqwt_skewnessValue_dec_36,tqwt_kurtosisValue_dec_1,tqwt_kurtosisValue_dec_2,tqwt_kurtosisValue_dec_3,tqwt_kurtosisValue_dec_4,tqwt_kurtosisValue_dec_5,tqwt_kurtosisValue_dec_6,tqwt_kurtosisValue_dec_7,tqwt_kurtosisValue_dec_8,tqwt_kurtosisValue_dec_9,tqwt_kurtosisValue_dec_10,tqwt_kurtosisValue_dec_11,tqwt_kurtosisValue_dec_12,tqwt_kurtosisValue_dec_13,tqwt_kurtosisValue_dec_14,tqwt_kurtosisValue_dec_15,tqwt_kurtosisValue_dec_16,tqwt_kurtosisValue_dec_17,tqwt_kurtosisValue_dec_18,tqwt_kurtosisValue_dec_19,tqwt_kurtosisValue_dec_20,tqwt_kurtosisValue_dec_21,tqwt_kurtosisValue_dec_22,tqwt_kurtosisValue_dec_23,tqwt_kurtosisValue_dec_24,tqwt_kurtosisValue_dec_25,tqwt_kurtosisValue_dec_26,tqwt_kurtosisValue_dec_27,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,0.000018,0.00067,0.00129,0.00200,0.05883,0.517,0.03011,0.03496,0.04828,0.09034,0.970805,0.036223,18.995,69.997496,76.088046,72.465512,539.342735,1031.849040,2447.162183,3655.054806,101.092218,83.147440,255.214830,396.643631,0.77778,11.7245,2.8277,1.17300,0.265120,0.083127,...,0.071728,0.010352,-2.730300,66.5007,36.9934,26.3508,51.9577,21.5451,8.2488,8.0024,6.9635,6.9189,5.0622,4.6068,7.7218,2.7224,2.4171,2.9383,4.2077,3.1541,2.8531,2.7496,2.1550,2.9457,2.1993,1.9830,1.8314,2.0062,1.6058,1.5466,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,0.000016,0.00052,0.00112,0.00157,0.05516,0.502,0.02320,0.03675,0.06195,0.06961,0.984322,0.017974,21.497,67.415903,73.046374,71.528945,564.363614,1016.367294,2383.565201,3498.681572,58.465428,86.487292,248.357127,218.229722,0.81250,13.8284,2.8908,1.02210,0.220040,0.127410,...,0.729330,0.780410,5.229400,8643.9860,3962.0554,2976.2411,4329.0607,4005.1329,1127.6762,116.5331,20.2332,13.6395,12.5514,13.6965,60.8974,6.2958,3.0323,3.4039,4.3794,2.4329,2.0585,2.1839,2.2061,3.0999,1.9824,1.6227,1.5783,2.0470,1.5772,1.5530,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,0.000015,0.00057,0.00111,0.00171,0.09902,0.897,0.05094,0.06497,0.07772,0.15282,0.974846,0.026313,17.651,62.661706,71.633549,68.086583,548.444604,1032.406341,2357.826954,3678.128717,160.387771,54.685168,151.694847,84.240339,0.81818,26.9273,2.6975,0.84951,0.157560,0.116890,...,0.269510,-0.005522,0.350540,29.2717,32.4971,38.9453,85.1480,62.5132,9.7308,6.8890,6.1357,5.4926,3.9078,4.0864,6.9828,3.4411,3.5173,3.7204,4.2212,3.4881,3.4851,3.3007,2.0427,3.1436,2.1203,1.6627,1.6731,3.2597,1.5921,1.5399,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,0.000046,0.00149,0.00268,0.00446,0.05451,0.527,0.02395,0.02857,0.04462,0.07185,0.968343,0.042003,19.865,76.306989,81.000749,79.190593,819.529588,1201.813897,3154.035654,4122.163933,238.667052,191.984916,573.752909,526.147599,0.98548,139.5744,1.6961,0.83405,0.172950,0.147370,...,0.366920,-0.492650,0.191640,591.0116,65.3827,53.9852,45.4458,34.6650,88.3259,46.6869,23.2911,19.2022,42.3957,100.3649,7.1967,6.3892,6.5496,6.0264,4.7656,4.8909,4.2531,3.0295,2.0362,1.8478,2.5776,2.2064,1.9491,1.9120,1.8829,6.9761,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4

In [ ]:
#남녀 한번 세어보자
df['gender'].value_counts()

1    390
0    366
Name: gender, dtype: int64

In [ ]:
gender = pd.get_dummies(df.loc[:, 'gender'])
# pd.get_dummies(df)# 원핫인코딩 실행

In [ ]:
gender = gender.rename(columns={0: 'female', 1: 'male'})
gender

,female,male
0,0,1
1,0,1
2,0,1
3,1,0
4,1,0
...,...,...
751,1,0
752,1,0
753,1,0
754,1,0


In [ ]:
df = pd.concat([gender, df], axis=1)
df

,female,male,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,rapJitter,ppq5Jitter,ddpJitter,locShimmer,locDbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer,meanAutoCorrHarmonicity,meanNoiseToHarmHarmonicity,meanHarmToNoiseHarmonicity,minIntensity,maxIntensity,meanIntensity,f1,f2,f3,f4,b1,b2,b3,b4,GQ_prc5_95,GQ_std_cycle_open,GQ_std_cycle_closed,GNE_mean,...,tqwt_skewnessValue_dec_34,tqwt_skewnessValue_dec_35,tqwt_skewnessValue_dec_36,tqwt_kurtosisValue_dec_1,tqwt_kurtosisValue_dec_2,tqwt_kurtosisValue_dec_3,tqwt_kurtosisValue_dec_4,tqwt_kurtosisValue_dec_5,tqwt_kurtosisValue_dec_6,tqwt_kurtosisValue_dec_7,tqwt_kurtosisValue_dec_8,tqwt_kurtosisValue_dec_9,tqwt_kurtosisValue_dec_10,tqwt_kurtosisValue_dec_11,tqwt_kurtosisValue_dec_12,tqwt_kurtosisValue_dec_13,tqwt_kurtosisValue_dec_14,tqwt_kurtosisValue_dec_15,tqwt_kurtosisValue_dec_16,tqwt_kurtosisValue_dec_17,tqwt_kurtosisValue_dec_18,tqwt_kurtosisValue_dec_19,tqwt_kurtosisValue_dec_20,tqwt_kurtosisValue_dec_21,tqwt_kurtosisValue_dec_22,tqwt_kurtosisValue_dec_23,tqwt_kurtosisValue_dec_24,tqwt_kurtosisValue_dec_25,tqwt_kurtosisValue_dec_26,tqwt_kurtosisValue_dec_27,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,0.000018,0.00067,0.00129,0.00200,0.05883,0.517,0.03011,0.03496,0.04828,0.09034,0.970805,0.036223,18.995,69.997496,76.088046,72.465512,539.342735,1031.849040,2447.162183,3655.054806,101.092218,83.147440,255.214830,396.643631,0.77778,11.7245,2.8277,1.17300,...,0.071728,0.010352,-2.730300,66.5007,36.9934,26.3508,51.9577,21.5451,8.2488,8.0024,6.9635,6.9189,5.0622,4.6068,7.7218,2.7224,2.4171,2.9383,4.2077,3.1541,2.8531,2.7496,2.1550,2.9457,2.1993,1.9830,1.8314,2.0062,1.6058,1.5466,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,0.000016,0.00052,0.00112,0.00157,0.05516,0.502,0.02320,0.03675,0.06195,0.06961,0.984322,0.017974,21.497,67.415903,73.046374,71.528945,564.363614,1016.367294,2383.565201,3498.681572,58.465428,86.487292,248.357127,218.229722,0.81250,13.8284,2.8908,1.02210,...,0.729330,0.780410,5.229400,8643.9860,3962.0554,2976.2411,4329.0607,4005.1329,1127.6762,116.5331,20.2332,13.6395,12.5514,13.6965,60.8974,6.2958,3.0323,3.4039,4.3794,2.4329,2.0585,2.1839,2.2061,3.0999,1.9824,1.6227,1.5783,2.0470,1.5772,1.5530,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,0.000015,0.00057,0.00111,0.00171,0.09902,0.897,0.05094,0.06497,0.07772,0.15282,0.974846,0.026313,17.651,62.661706,71.633549,68.086583,548.444604,1032.406341,2357.826954,3678.128717,160.387771,54.685168,151.694847,84.240339,0.81818,26.9273,2.6975,0.84951,...,0.269510,-0.005522,0.350540,29.2717,32.4971,38.9453,85.1480,62.5132,9.7308,6.8890,6.1357,5.4926,3.9078,4.0864,6.9828,3.4411,3.5173,3.7204,4.2212,3.4881,3.4851,3.3007,2.0427,3.1436,2.1203,1.6627,1.6731,3.2597,1.5921,1.5399,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,0.000046,0.00149,0.00268,0.00446,0.05451,0.527,0.02395,0.02857,0.04462,0.07185,0.968343,0.042003,19.865,76.306989,81.000749,79.190593,819.529588,1201.813897,3154.035654,4122.163933,238.667052,191.984916,573.752909,526.147599,0.98548,139.5744,1.6961,0.83405,...,0.366920,-0.492650,0.191640,591.0116,65.3827,53.9852,45.4458,34.6650,88.3259,46.6869,23.2911,19.2022,42.3957,100.3649,7.1967,6.3892,6.5496,6.0264,4.7656,4.8909,4.2531,3.0295,2.0362,1.8478,2.5776,2.2064,1.9491,1.9120,1.8829,6.9761,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,1,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.0053

In [ ]:
df = df.drop(columns=['gender'])
df

,female,male,id,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,rapJitter,ppq5Jitter,ddpJitter,locShimmer,locDbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer,meanAutoCorrHarmonicity,meanNoiseToHarmHarmonicity,meanHarmToNoiseHarmonicity,minIntensity,maxIntensity,meanIntensity,f1,f2,f3,f4,b1,b2,b3,b4,GQ_prc5_95,GQ_std_cycle_open,GQ_std_cycle_closed,GNE_mean,GNE_std,...,tqwt_skewnessValue_dec_34,tqwt_skewnessValue_dec_35,tqwt_skewnessValue_dec_36,tqwt_kurtosisValue_dec_1,tqwt_kurtosisValue_dec_2,tqwt_kurtosisValue_dec_3,tqwt_kurtosisValue_dec_4,tqwt_kurtosisValue_dec_5,tqwt_kurtosisValue_dec_6,tqwt_kurtosisValue_dec_7,tqwt_kurtosisValue_dec_8,tqwt_kurtosisValue_dec_9,tqwt_kurtosisValue_dec_10,tqwt_kurtosisValue_dec_11,tqwt_kurtosisValue_dec_12,tqwt_kurtosisValue_dec_13,tqwt_kurtosisValue_dec_14,tqwt_kurtosisValue_dec_15,tqwt_kurtosisValue_dec_16,tqwt_kurtosisValue_dec_17,tqwt_kurtosisValue_dec_18,tqwt_kurtosisValue_dec_19,tqwt_kurtosisValue_dec_20,tqwt_kurtosisValue_dec_21,tqwt_kurtosisValue_dec_22,tqwt_kurtosisValue_dec_23,tqwt_kurtosisValue_dec_24,tqwt_kurtosisValue_dec_25,tqwt_kurtosisValue_dec_26,tqwt_kurtosisValue_dec_27,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,0.000018,0.00067,0.00129,0.00200,0.05883,0.517,0.03011,0.03496,0.04828,0.09034,0.970805,0.036223,18.995,69.997496,76.088046,72.465512,539.342735,1031.849040,2447.162183,3655.054806,101.092218,83.147440,255.214830,396.643631,0.77778,11.7245,2.8277,1.17300,0.265120,...,0.071728,0.010352,-2.730300,66.5007,36.9934,26.3508,51.9577,21.5451,8.2488,8.0024,6.9635,6.9189,5.0622,4.6068,7.7218,2.7224,2.4171,2.9383,4.2077,3.1541,2.8531,2.7496,2.1550,2.9457,2.1993,1.9830,1.8314,2.0062,1.6058,1.5466,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,0.000016,0.00052,0.00112,0.00157,0.05516,0.502,0.02320,0.03675,0.06195,0.06961,0.984322,0.017974,21.497,67.415903,73.046374,71.528945,564.363614,1016.367294,2383.565201,3498.681572,58.465428,86.487292,248.357127,218.229722,0.81250,13.8284,2.8908,1.02210,0.220040,...,0.729330,0.780410,5.229400,8643.9860,3962.0554,2976.2411,4329.0607,4005.1329,1127.6762,116.5331,20.2332,13.6395,12.5514,13.6965,60.8974,6.2958,3.0323,3.4039,4.3794,2.4329,2.0585,2.1839,2.2061,3.0999,1.9824,1.6227,1.5783,2.0470,1.5772,1.5530,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,0.000015,0.00057,0.00111,0.00171,0.09902,0.897,0.05094,0.06497,0.07772,0.15282,0.974846,0.026313,17.651,62.661706,71.633549,68.086583,548.444604,1032.406341,2357.826954,3678.128717,160.387771,54.685168,151.694847,84.240339,0.81818,26.9273,2.6975,0.84951,0.157560,...,0.269510,-0.005522,0.350540,29.2717,32.4971,38.9453,85.1480,62.5132,9.7308,6.8890,6.1357,5.4926,3.9078,4.0864,6.9828,3.4411,3.5173,3.7204,4.2212,3.4881,3.4851,3.3007,2.0427,3.1436,2.1203,1.6627,1.6731,3.2597,1.5921,1.5399,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,1,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,0.000046,0.00149,0.00268,0.00446,0.05451,0.527,0.02395,0.02857,0.04462,0.07185,0.968343,0.042003,19.865,76.306989,81.000749,79.190593,819.529588,1201.813897,3154.035654,4122.163933,238.667052,191.984916,573.752909,526.147599,0.98548,139.5744,1.6961,0.83405,0.172950,...,0.366920,-0.492650,0.191640,591.0116,65.3827,53.9852,45.4458,34.6650,88.3259,46.6869,23.2911,19.2022,42.3957,100.3649,7.1967,6.3892,6.5496,6.0264,4.7656,4.8909,4.2531,3.0295,2.0362,1.8478,2.5776,2.2064,1.9491,1.9120,1.8829,6.9761,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,1,0.32790,0.79782,0.53028,236,2

### ID중복방지(차선책 실행시 실행X)
같은 ID의 데이터가 train set과 test set에 모두 들어가는 것을 방지하기

그런데 이거 10 fold로 어떻게 하는지 모르겠다...

In [ ]:
RS = 456

from sklearn.model_selection import train_test_split
train, test = train_test_split(range(252),
                               stratify=df[['id', 'class']].drop_duplicates()['class'].tolist(),
                               random_state=RS,
                               test_size=0.2) # train:test = 8:2

In [ ]:
df_train = df[df['id'].isin(train)]
df_test = df[df['id'].isin(test)]

In [ ]:
df_train = df_train.drop(columns=['id'])
df_test = df_test.drop(columns=['id'])

In [ ]:
X_train = df_train.iloc[:, :-1]
X_test = df_test.iloc[:, :-1]
y_train = df_train.iloc[:, -1]
y_test = df_test.iloc[:, -1]

### ID중복제거(차선책)
row기준으로 중복된 id를 다 제거해보자. 구현력의 한계 때문에 두는 차선책.

In [ ]:
df = df.drop_duplicates(subset=['id'])

In [ ]:
df = df.drop(columns=['id'])

## LGBM중복방지(acc: 0.8824 / AUC: 0.7777)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score
from lightgbm import LGBMClassifier

In [ ]:
model = LGBMClassifier(random_state=RS, device='gpu')

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred) 
auc = roc_auc_score(y_test, y_pred)
print("Accuracy : {}\nAUC : {}".format(acc, auc))

Accuracy : 0.8823529411764706
AUC : 0.7776653171390013


##LGBM중복제거 & 10-fold cv(acc: 0.8424 / AUC 0.7256)

In [ ]:
# StratifiedKFold guerantees uniform proportion of target values for each fold
from sklearn.model_selection import StratifiedKFold

RS = 456
# shuffle: random shuffle data, random_state: key of random shuffle
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=RS)

In [ ]:
target_col = 'class'
unused_cols = []

In [ ]:
X = df[[col for col in df.columns if col not in unused_cols + [target_col]]].values
y = df[target_col].values

In [ ]:
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
accs = []
aucs = []

model = LGBMClassifier(random_state=RS, device='gpu')

for fold, (train_indice, test_indice) in enumerate(kfold.split(X, y)):
    X_train = X[train_indice]
    y_train = y[train_indice]
    X_test = X[test_indice]
    y_test = y[test_indice]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    accs.append(acc)
    aucs.append(auc)
    print("[Fold {}] AUC : {}".format(fold, aucs[fold]))

print('Mean accuracy : {}'.format(np.mean(accs)))
print('Mean AUC : {}'.format(np.mean(aucs))) 

[Fold 0] AUC : 0.5902255639097744
[Fold 1] AUC : 0.8045112781954888
[Fold 2] AUC : 0.6616541353383458
[Fold 3] AUC : 0.6428571428571428
[Fold 4] AUC : 0.7236842105263157
[Fold 5] AUC : 0.6666666666666667
[Fold 6] AUC : 0.8333333333333334
[Fold 7] AUC : 0.8333333333333334
[Fold 8] AUC : 0.75
[Fold 9] AUC : 0.75
Mean accuracy : 0.8423846153846155
Mean AUC : 0.7256265664160401


## LGBM 중복제거 & Shap & 10fold 적용(acc: 0.7307 / AUC: 0.6192)

#### BorutaShap 구축

In [ ]:
!pip install BorutaShap

In [ ]:
from BorutaShap import BorutaShap
from lightgbm import LGBMClassifier

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
RS = 456

model = LGBMClassifier(random_state=RS, device='gpu')

# if classification is False it is a Regression problem
Feature_Selector = BorutaShap(model=model,
                              importance_measure='shap',
                              classification=True)

Feature_Selector.fit(X=df.iloc[:, :-1], y=df['class'],
                     n_trials=100, sample=False,
                     train_or_test = 'test', normalize=True,
                     verbose=True)

100%|██████████| 100/100 [01:02<00:00,  1.61it/s]

3 attributes confirmed important: ['std_8th_delta', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_15']
751 attributes confirmed unimportant: ['det_LT_TKEO_std_5_coef', 'det_LT_entropy_shannon_6_coef', 'GNE_NSR_TKEO', 'tqwt_minValue_dec_25', 'locDbShimmer', 'tqwt_energy_dec_7', 'mean_MFCC_9th_coef', 'app_LT_entropy_shannon_4_coef', 'VFER_SNR_TKEO', 'tqwt_maxValue_dec_15', 'tqwt_kurtosisValue_dec_29', 'tqwt_skewnessValue_dec_14', 'det_LT_TKEO_mean_1_coef', 'tqwt_minValue_dec_21', 'tqwt_minValue_dec_13', 'tqwt_kurtosisValue_dec_17', 'std_12th_delta', 'app_det_TKEO_mean_6_coef', 'tqwt_skewnessValue_dec_35', 'tqwt_meanValue_dec_11', 'tqwt_maxValue_dec_10', 'tqwt_maxValue_dec_24', 'tqwt_entropy_shannon_dec_1', 'tqwt_TKEO_mean_dec_6', 'det_LT_entropy_log_3_coef', 'det_entropy_shannon_9_coef', 'GNE_mean', 'tqwt_minValue_dec_19', 'tqwt_kurtosisValue_dec_10', 'std_3rd_delta_delta', 'tqwt_stdValue_dec_7', 'tqwt_medianValue_dec_16', 'tqwt_energy_dec_17', 'Ed_4_coef', 'det_TKEO_mean_9_coef'

중요한 피쳐 3개:

['std_8th_delta', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_15']

불확실한 피쳐 0개:

나머지는 안중요

#### BorutaShap 적용 10-fold cv

In [ ]:
# StratifiedKFold guerantees uniform proportion of target values for each fold
from sklearn.model_selection import StratifiedKFold

RS = 456
# shuffle: random shuffle data, random_state: key of random shuffle
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=RS)

X = df[['std_8th_delta', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_15']].values
y = df['class'].values

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score  # for printing score

from lightgbm import LGBMClassifier

In [ ]:
accs = []
aucs = []

model = LGBMClassifier(random_state=RS, device='gpu')

for fold, (train_indice, test_indice) in enumerate(kfold.split(X, y)):
    X_train = X[train_indice]
    y_train = y[train_indice]
    X_test = X[test_indice]
    y_test = y[test_indice]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    accs.append(acc)
    aucs.append(auc)
    print("[Fold {}] AUC : {}".format(fold, aucs[fold]))

print('Mean accuracy : {}'.format(np.mean(accs)))
print('Mean AUC : {}'.format(np.mean(aucs))) 

[Fold 0] AUC : 0.5902255639097744
[Fold 1] AUC : 0.5375939849624061
[Fold 2] AUC : 0.5112781954887218
[Fold 3] AUC : 0.706766917293233
[Fold 4] AUC : 0.5877192982456141
[Fold 5] AUC : 0.6973684210526316
[Fold 6] AUC : 0.6140350877192983
[Fold 7] AUC : 0.6973684210526316
[Fold 8] AUC : 0.6666666666666667
[Fold 9] AUC : 0.5833333333333335
Mean accuracy : 0.7307564102564102
Mean AUC : 0.6192355889724311


In [ ]:
df_train_shap = df_train[['std_8th_delta', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_15']]
df_test_shap = df_test[['std_8th_delta', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_15']]

X_train = df_train_shap
X_test = df_test_shap
y_train = df_train.iloc[:, -1]
y_test = df_test.iloc[:, -1]

In [ ]:
model = LGBMClassifier(random_state=RS, device='gpu')

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred) 
auc = roc_auc_score(y_test, y_pred)
print("Accuracy : {}\nAUC : {}".format(acc, auc))

Accuracy : 0.9084967320261438
AUC : 0.8289473684210527


## XGBoost중복방지(acc: 0.7974 / AUC: 0.7038)

In [ ]:
from xgboost import XGBRFClassifier

model = XGBRFClassifier(random_state=RS)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred) 
auc = roc_auc_score(y_test, y_pred)
print("Accuracy : {}\nAUC : {}".format(acc, auc))

Accuracy : 0.7973856209150327
AUC : 0.703778677462888


## XGBoost중복제거 & 10-fold cv(acc: 0.8232 / AUC: 0.6989)

In [ ]:
# StratifiedKFold guerantees uniform proportion of target values for each fold
from sklearn.model_selection import StratifiedKFold
import numpy as np
from xgboost import XGBRFClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

RS = 456
# shuffle: random shuffle data, random_state: key of random shuffle
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=RS)

In [ ]:
target_col = 'class'
unused_cols = []

In [ ]:
X = df[[col for col in df.columns if col not in unused_cols + [target_col]]].values
y = df[target_col].values

In [ ]:
accs = []
aucs = []

model = XGBRFClassifier(random_state=RS)

for fold, (train_indice, test_indice) in enumerate(kfold.split(X, y)):
    X_train = X[train_indice]
    y_train = y[train_indice]
    X_test = X[test_indice]
    y_test = y[test_indice]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    accs.append(acc)
    aucs.append(auc)
    print("[Fold {}] AUC : {}".format(fold, aucs[fold]))

print('Mean accuracy : {}'.format(np.mean(accs)))
print('Mean AUC : {}'.format(np.mean(aucs))) 

[Fold 0] AUC : 0.5902255639097744
[Fold 1] AUC : 0.7142857142857143
[Fold 2] AUC : 0.6090225563909775
[Fold 3] AUC : 0.5451127819548872
[Fold 4] AUC : 0.6973684210526316
[Fold 5] AUC : 0.6666666666666667
[Fold 6] AUC : 0.75
[Fold 7] AUC : 0.8333333333333334
[Fold 8] AUC : 0.75
[Fold 9] AUC : 0.8333333333333334
Mean accuracy : 0.8231666666666667
Mean AUC : 0.6989348370927317


## CatBoost중복방지(acc: 0.8889 / AUC: 0.7905)

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(random_state=RS, task_type='GPU', verbose=0)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred) 
auc = roc_auc_score(y_test, y_pred)
print("Accuracy : {}\nAUC : {}".format(acc, auc))

Accuracy : 0.8888888888888888
AUC : 0.7904858299595141


## CatBoost중복제거 & 10-fold cv(acc: 0.8264 / AUC: 0.6257)

In [ ]:
# StratifiedKFold guerantees uniform proportion of target values for each fold
from sklearn.model_selection import StratifiedKFold
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

RS = 456
# shuffle: random shuffle data, random_state: key of random shuffle
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=RS)

In [ ]:
target_col = 'class'
unused_cols = []

In [ ]:
X = df[[col for col in df.columns if col not in unused_cols + [target_col]]].values
y = df[target_col].values

In [ ]:
accs = []
aucs = []

model = CatBoostClassifier(random_state=RS, task_type='GPU', verbose=0)

for fold, (train_indice, test_indice) in enumerate(kfold.split(X, y)):
    X_train = X[train_indice]
    y_train = y[train_indice]
    X_test = X[test_indice]
    y_test = y[test_indice]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    accs.append(acc)
    aucs.append(auc)
    print("[Fold {}] AUC : {}".format(fold, aucs[fold]))

print('Mean accuracy : {}'.format(np.mean(accs)))
print('Mean AUC : {}'.format(np.mean(aucs))) 

[Fold 0] AUC : 0.6165413533834586
[Fold 1] AUC : 0.6879699248120301
[Fold 2] AUC : 0.5902255639097744
[Fold 3] AUC : 0.6428571428571428
[Fold 4] AUC : 0.7236842105263157
[Fold 5] AUC : 0.5570175438596491
[Fold 6] AUC : 0.8333333333333334
[Fold 7] AUC : 0.8333333333333334
[Fold 8] AUC : 0.6388888888888888
[Fold 9] AUC : 0.8333333333333334
Mean accuracy : 0.826371794871795
Mean AUC : 0.6957184628237261


### Borutashap 적용

In [ ]:
df_train_shap = df_train[['tqwt_entropy_log_dec_12', 'tqwt_minValue_dec_17', 'tqwt_energy_dec_12', 'DFA', 'std_MFCC_2nd_coef', 'mean_1st_delta_delta', 'std_6th_delta_delta', 'mean_MFCC_2nd_coef', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_17', 'tqwt_energy_dec_25', 'tqwt_TKEO_mean_dec_25', 'tqwt_TKEO_std_dec_25', 'tqwt_entropy_log_dec_27', 'tqwt_TKEO_std_dec_11', 'std_11th_delta_delta', 'tqwt_entropy_log_dec_33', 'IMF_NSR_TKEO', 'tqwt_skewnessValue_dec_25', 'std_delta_log_energy', 'tqwt_energy_dec_26', 'std_8th_delta_delta', 'std_7th_delta_delta', 'tqwt_TKEO_std_dec_20', 'tqwt_minValue_dec_13', 'tqwt_skewnessValue_dec_12', 'tqwt_energy_dec_6', 'mean_MFCC_1st_coef', 'tqwt_kurtosisValue_dec_20', 'mean_MFCC_6th_coef', 'tqwt_meanValue_dec_36', 'tqwt_entropy_log_dec_35', 'mean_2nd_delta', 'tqwt_stdValue_dec_18', 'tqwt_meanValue_dec_5', 'minIntensity']
df_test_shap = df_test[['tqwt_entropy_log_dec_12', 'tqwt_minValue_dec_17', 'tqwt_energy_dec_12', 'DFA', 'std_MFCC_2nd_coef', 'mean_1st_delta_delta', 'std_6th_delta_delta', 'mean_MFCC_2nd_coef', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_17', 'tqwt_energy_dec_25', 'tqwt_TKEO_mean_dec_25', 'tqwt_TKEO_std_dec_25', 'tqwt_entropy_log_dec_27', 'tqwt_TKEO_std_dec_11', 'std_11th_delta_delta', 'tqwt_entropy_log_dec_33', 'IMF_NSR_TKEO', 'tqwt_skewnessValue_dec_25', 'std_delta_log_energy', 'tqwt_energy_dec_26', 'std_8th_delta_delta', 'std_7th_delta_delta', 'tqwt_TKEO_std_dec_20', 'tqwt_minValue_dec_13', 'tqwt_skewnessValue_dec_12', 'tqwt_energy_dec_6', 'mean_MFCC_1st_coef', 'tqwt_kurtosisValue_dec_20', 'mean_MFCC_6th_coef', 'tqwt_meanValue_dec_36', 'tqwt_entropy_log_dec_35', 'mean_2nd_delta', 'tqwt_stdValue_dec_18', 'tqwt_meanValue_dec_5', 'minIntensity', 'tqwt_skewnessValue_dec_36', 'tqwt_energy_dec_27', 'tqwt_kurtosisValue_dec_18', 'tqwt_TKEO_std_dec_19', 'tqwt_energy_dec_11', 'tqwt_energy_dec_7', 'std_6th_delta', 'mean_delta_delta_log_energy', 'tqwt_entropy_log_dec_16', 'tqwt_entropy_log_dec_32', 'tqwt_entropy_shannon_dec_25', 'tqwt_medianValue_dec_1', 'tqwt_kurtosisValue_dec_36']]

X_train = df_train[]
X_test = df_test[]
y_train = df_train.iloc[:, -1]
y_test = df_test.iloc[:, -1]

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(random_state=RS, task_type='GPU', verbose=0)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred) 
auc = roc_auc_score(y_test, y_pred)
print("Accuracy : {}\nAUC : {}".format(acc, auc))

## CatBoost 중복제거 & Shap & 10fold 적용(acc: 0.7738 / AUC: 0.6749)

#### BorutaShap 구축

In [ ]:
!pip install BorutaShap

In [ ]:
from BorutaShap import BorutaShap
from catboost import CatBoostClassifier

In [ ]:
RS = 456

model = CatBoostClassifier(random_state=RS, task_type='GPU')

# if classification is False it is a Regression problem
Feature_Selector = BorutaShap(model=model,
                              importance_measure='shap',
                              classification=True)

Feature_Selector.fit(X=df.iloc[:, :-1], y=df['class'],
                     n_trials=100, sample=False,
                     train_or_test = 'test', normalize=True,
                     verbose=True)

100%|██████████| 100/100 [1:02:18<00:00, 37.38s/it]

2 attributes confirmed important: ['std_delta_delta_log_energy', 'tqwt_entropy_log_dec_27']
751 attributes confirmed unimportant: ['det_LT_TKEO_std_5_coef', 'det_LT_entropy_shannon_6_coef', 'GNE_NSR_TKEO', 'tqwt_minValue_dec_25', 'locDbShimmer', 'tqwt_energy_dec_7', 'mean_MFCC_9th_coef', 'app_LT_entropy_shannon_4_coef', 'VFER_SNR_TKEO', 'tqwt_maxValue_dec_15', 'tqwt_kurtosisValue_dec_29', 'tqwt_skewnessValue_dec_14', 'det_LT_TKEO_mean_1_coef', 'tqwt_minValue_dec_21', 'tqwt_minValue_dec_13', 'tqwt_kurtosisValue_dec_17', 'std_12th_delta', 'app_det_TKEO_mean_6_coef', 'tqwt_skewnessValue_dec_35', 'tqwt_meanValue_dec_11', 'tqwt_maxValue_dec_10', 'tqwt_maxValue_dec_24', 'tqwt_entropy_shannon_dec_1', 'tqwt_TKEO_mean_dec_6', 'det_LT_entropy_log_3_coef', 'det_entropy_shannon_9_coef', 'GNE_mean', 'tqwt_minValue_dec_19', 'tqwt_kurtosisValue_dec_10', 'std_3rd_delta_delta', 'tqwt_stdValue_dec_7', 'tqwt_medianValue_dec_16', 'tqwt_energy_dec_17', 'Ed_4_coef', 'det_TKEO_mean_9_coef', 'numPeriodsPulses

중요한 피쳐 2개:

['std_delta_delta_log_energy', 'tqwt_entropy_log_dec_27']

불확실한 피쳐 1개:

['tqwt_medianValue_dec_14']

나머지 751개는 안중요

#### BorutaShap 적용 10-fold cv

In [ ]:
# StratifiedKFold guerantees uniform proportion of target values for each fold
from sklearn.model_selection import StratifiedKFold

RS = 456
# shuffle: random shuffle data, random_state: key of random shuffle
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=RS)

X = df[['std_delta_delta_log_energy', 'tqwt_entropy_log_dec_27', 'tqwt_medianValue_dec_14']].values
y = df['class'].values

In [ ]:
!pip install catboost

     |████████████████████████████████| 66.3MB 44kB/s 


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score  # for printing score

from catboost import CatBoostClassifier

In [ ]:
accs = []
aucs = []

model = CatBoostClassifier(random_state=RS, task_type='GPU', verbose=0)

for fold, (train_indice, test_indice) in enumerate(kfold.split(X, y)):
    X_train = X[train_indice]
    y_train = y[train_indice]
    X_test = X[test_indice]
    y_test = y[test_indice]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    accs.append(acc)
    aucs.append(auc)
    print("[Fold {}] AUC : {}".format(fold, aucs[fold]))

print('Mean accuracy : {}'.format(np.mean(accs)))
print('Mean AUC : {}'.format(np.mean(aucs))) 

[Fold 0] AUC : 0.706766917293233
[Fold 1] AUC : 0.706766917293233
[Fold 2] AUC : 0.9473684210526316
[Fold 3] AUC : 0.6140350877192983
[Fold 4] AUC : 0.6973684210526316
[Fold 5] AUC : 0.6140350877192983
[Fold 6] AUC : 0.6447368421052632
[Fold 7] AUC : 0.5877192982456141
[Fold 8] AUC : 0.6865079365079365
[Fold 9] AUC : 0.5436507936507936
Mean accuracy : 0.7738461538461539
Mean AUC : 0.6748955722639933


In [ ]:
df_train_shap = df_train[['std_8th_delta', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_15']]
df_test_shap = df_test[['std_8th_delta', 'std_delta_delta_log_energy', 'tqwt_kurtosisValue_dec_15']]

X_train = df_train_shap
X_test = df_test_shap
y_train = df_train.iloc[:, -1]
y_test = df_test.iloc[:, -1]

In [ ]:
model = LGBMClassifier(random_state=RS, device='gpu')

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred) 
auc = roc_auc_score(y_test, y_pred)
print("Accuracy : {}\nAUC : {}".format(acc, auc))

Accuracy : 0.9084967320261438
AUC : 0.8289473684210527
